<a href="https://colab.research.google.com/github/meizhong986/WhisperJAV/blob/main/WhisperJAV_v0_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# For credits, and instructions please see the bottom of this notebook


You can run this notebook by selecting Runtime from top menu | Run All. Or press Ctrl+F9.

Please also see instructions below for connecting your Google Drive with this notebook and path to your audio folders.

In [ ]:
#@markdown **GPU VRAM and CPU RAM check** (Ideal: T4 and High-RAM)
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  #print(gpu_info)
  print("Using GPU")

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime\n')
else:
  print('You are using a high-RAM runtime!\n')


In [ ]:
#@markdown **Setup and Installation**
#@markdown *Takes roughly 90 seconds*

%%capture
!sudo apt-get install sox libsox-fmt-mp3 libsndfile1
! add-apt-repository -y ppa:savoury1/ffmpeg4
! apt-get -qq install -y ffmpeg
#!ffmpeg -version

%pip install -q ffmpeg-python srt deepl faster-whisper
%pip install pysrt
%pip install nagisa



In [ ]:
from dynet_viz import new_builder_num
#@markdown **Global Parameters and Definitions [v0.6]**



TQDM_FORMAT = "{desc}: {percentage:3.1f}% |{bar}| {n:.2f}/{total:.2f} [{elapsed}<{remaining}, {rate:.2f} audio s / real time s]"

PUNCT_MATCH = ["。", "、", ",", ".", "〜", "！", "!", "？", "?", "-"]
REMOVE_QUOTES = dict.fromkeys(map(ord, '"„“‟”＂「」'), None)
GARBAGE_LIST2 = [
	"a",
	"aa",
	"ah",
	"ahh",
	"h",
	"ha",
	"haa",
	"hah",
	"haha",
	"hahaha",
	"hm",
	"hmm",
	"huh",
	"m",
	"mh",
	"mm",
	"mmh",
	"mmm",
	"o",
	"oh",
  "I'm sorry.",
  "I'm sorry",
  "I'm hungry.",
  "I'm hungry"
  ]

NEED_CONTEXT_LINES = [
	"feelsgod",
	"godbye",
	"godnight",
	"thankyou",
]

clean_text = lambda text: (text
	.replace(".", "")
	.replace(",", "")
	.replace(":", "")
	.replace(";", "")
	.replace("!", "")
	.replace("?", "")
	.replace("-", " ")
	.replace("  ", " ")
	.replace("  ", " ")
	.replace("  ", " ")
	.lower()
	.replace("that feels", "feels")
	.replace("it feels", "feels")
	.replace("feels good", "feelsgood")
	.replace("good bye", "goodbye")
	.replace("good night", "goodnight")
	.replace("thank you", "thankyou")
	.replace("aaaaaa", "a")
	.replace("aaaa", "a")
	.replace("aa", "a")
	.replace("aa", "a")
	.replace("mmmmmm", "m")
	.replace("mmmm", "m")
	.replace("mm", "m")
	.replace("mm", "m")
	.replace("hhhhhh", "h")
	.replace("hhhh", "h")
	.replace("hh", "h")
	.replace("hh", "h")
	.replace("oooooo", "o")
	.replace("oooo", "o")
	.replace("oo", "o")
	.replace("oo", "o")
)

TO_LANGUAGE_CODE = { # from https://github.com/openai/whisper/blob/main/whisper/tokenizer.py
	"afrikaans": "af",
	"chinese": "zh",
	"english": "en",
	"japanese": "ja",
	"korean": "ko"
}


suppress_low = [
    "Thank you",
    "Thanks for",
    "ike and ",
    "Bye.",
    "Bye!",
    "Bye bye!",
    "lease sub",
    "The end.",
    "視聴",
    "お腹の中さ",
    "近びも",
    "ペコペコ",
    "おばあちゃん",
    "た、た、た",
    "たたた",
    "また泣いて",
    "あら",
    "あんちゃん。",
    "あんちゃん",
    "寝てるの?",
    "寝てるの",
    "寝てる",
    "寝てる?",
    "寝たい",
    "ああ、寝、寝、",
    "ああ、寝、寝",
    "ああ、寝、",
    "ああ、寝",
    "あ、寝、寝、",
    "あ、寝、寝",
    "あ、寝、",
    "あ、寝",
    "ああ寝寝",
    "あ寝寝",
    "お前が寝たい",
    "えが",
    "プー",
    "気持ち悪い",
    "気持ち悪い?",
    "あー、にもじーの",
    "あーにもじーの",
]

suppress_high = [
    "subscribe",
    "my channel",
    "the channel",
    "our channel",
    "follow me on",
    "for watching",
    "thank you for watching",
    "for your viewing",
    "r viewing",
    "Amara",
    "next video",
    "full video",
    "translation by",
    "translated by",
    "see you next week",
    "I'm sorry.",
    "I'm hungry.",
    "I'm sorry",
    "I'm hungry",
    "おじいちゃん",
    "寝たらいいの?",
    "寝たらいいの",
    "あ、痛い",
    "痛い痛い",
    "痛い!",
    "痛い",
    "痛い?",
    "泣いてる",
    "お湯を沸騰させる",
    "鍋を温める",
    "鶏肉を入れて炒めます",
    "鶏肉が煮えたら、水を加えます",
    "鶏肉が煮えたら水を加えます",
    "ご視聴",
    "お腹がすっごい空きちゃった",
    "この動画を見てくれてありがとう!",
    "この動画を見てくれてありがとう",
    "視聴ありがとうございました",
    "見てくれてありがとう!",
    "見てくれてありがとう",
    "ごちそうさまでした",
    "ごちそうさまでした。",
    "お疲れ様でした",
    "お疲れ様",
    "お腹空いた",
    "お腹の音がする",
    "お腹いっぱい",
    "おやすみ",
    "おやすみなさい",
    "この後も、お互いの感情を感じることができます。",
    "この後も、お互いの感情を感じることができます",
    "お互いの感情を感じることができます。",
    "お互いの感情を感じることができます",
    "皆さんと一緒に楽しんでいってください。",
    "皆さんと一緒に楽しんでいってください",
    "チャンネル登録してね!",
    "チャンネル登録してね",
    "チャンネル登録をお願いいたします。",
    "チャンネル登録をお願いいたします",
    "チャンネル登録お願いします,",
    "チャンネル登録よろしくお願いします",
    "私のチャンネルはここまでです。",
    "私のチャンネルはここまでです",
    "私のチャンネルの他の動画も見てください。",
    "私のチャンネルの他の動画も見てください",
    "また、楽しい一日をお過ごしください。",
    "また、楽しい一日をお過ごしください",
    "また次の動画でお会いしましょう!",
    "また次の動画でお会いしましょう",
    "またね!",
    "またね",
    "【映像をご覧いただきありがとうございます】",
    "映像をご覧いただきありがとうございます",
    "ご覧いただきありがとうございます",
    "はじめしゃちょーエンディング",
    "【はじめしゃちょーエンディング】",
    "【はじめの泣き声】",
    "【はじめの笑い声】",
    "はじめの笑い声",
    "はじめの泣き声",
    "【はじめの咳】",
    "はじめの咳",
    "【はじめの悲鳴】",
    "はじめの悲鳴",
    "お腹鳴った",
    "【泣き声】",
    "泣き声",
    "【泣】",
    "泣",
    "【ED】",
    "ED",
    "【はじめの鼻息】",
    "はじめの鼻息",
    "【鳴き声】",
    "鳴き声",
    "娘の泣き声",
    "★ごきげんよう、また見てね★",
    "ごきげんよう、また見てね",
    "ご覧いただきありがとうございます。良い1日を!",
    "ご覧いただきありがとうございます。良い1日を",
    "お疲れ様です",
    "お疲れ様です。",
    "【早送り】",
    "早送り",
    "【赤ちゃんの泣き声】",
    "赤ちゃんの泣き声",
    "お腹が痛い",
    "私もお腹が痛い",
    "お腹が痛そうだった",
    "お腹すいた?",
    "お腹すいた",
    "お腹がすいた",
    "寝ますよ",
    "寝たら、お腹が痛くなってきた",
    "寝たらお腹が痛くなってきた",
    "1.5ページ",
    "2.5ページ",
    "3ページ",
    "4ページ",
    "5ページ",
    "pirate",
    "www",
    "wwwww",
    "hand",
    "in",
    "COOK",
    "🐕",
    "🐕🐕",
    "💦",
    "🐬",
    "🐾",
    "🐦",
    "★ごきげんよう、また見てね★",
    "ごきげんよう、また見てね",
    "おやつ",
    "★おうくん、おもちゃを取ったよ★",
    "おうくん、おもちゃを取ったよ",
    "おじいちゃんがおじいちゃんを救ってくれました。",
    "おじいちゃんがおじいちゃんを救ってくれました",
    "※この動画の字幕は視聴者によって作成されました。",
    "※この動画の字幕は視聴者によって作成されました",
    "この動画の字幕は視聴者によって作成されました。",
    "この動画の字幕は視聴者によって作成されました",
]

garbage_list = [
    "a",
    "aa",
    "ah",
    "ahh",
    "ha",
    "haa",
    "hah",
    "haha",
    "hahaha",
    "mmm",
    "mm",
    "m",
    "h",
    "o",
    "mh",
    "mmh",
    "hm",
    "hmm",
    "huh",
    "oh",
    "寝たらいいの?",
    "寝たらいいの",
    "寝てるの?",
    "寝てるの",
    "寝てる",
    "寝てる?",
    "寝たい",
    "お前が寝たい",
    "ああ、寝、寝、",
    "ああ、寝、寝",
    "ああ、寝、",
    "ああ、寝",
    "あ、寝、寝、",
    "あ、寝、寝",
    "あ、寝、",
    "あ、寝",
    "ああ寝寝",
    "あ寝寝",
    "あんちゃん。",
    "あんちゃん",
    "あら",
    "泣いてる",
    "また泣いて",
    "プー",
    "チャンネル登録してね!",
    "チャンネル登録してね",
    "チャンネル登録をお願いいたします。",
    "チャンネル登録をお願いいたします",
    "チャンネル登録お願いします,",
    "チャンネル登録よろしくお願いします",
    "私のチャンネルはここまでです。",
    "私のチャンネルはここまでです",
    "私のチャンネルの他の動画も見てください。",
    "私のチャンネルの他の動画も見てください",
    "また、楽しい一日をお過ごしください。",
    "また、楽しい一日をお過ごしください",
    "また次の動画でお会いしましょう!",
    "また次の動画でお会いしましょう",
    "またね!",
    "またね",
    "お湯を沸騰させる",
    "鍋を温める",
    "見てくれてありがとう!",
    "見てくれてありがとう",
    "鶏肉を入れて炒めます",
    "鶏肉が煮えたら、水を加えます",
    "鶏肉が煮えたら水を加えます",
    "【映像をご覧いただきありがとうございます】",
    "映像をご覧いただきありがとうございます",
    "この後も、お互いの感情を感じることができます。",
    "この後も、お互いの感情を感じることができます",
    "お互いの感情を感じることができます。",
    "お互いの感情を感じることができます",
    "皆さんと一緒に楽しんでいってください。",
    "皆さんと一緒に楽しんでいってください",
    "ご覧いただきありがとうございます",
    "この動画を見てくれてありがとう!",
    "この動画を見てくれてありがとう",
    "お腹がすっごい空きちゃった",
    "お腹がすいた",
    "お腹の音がする",
    "お腹いっぱい",
    "寝たら、お腹が痛くなってきた",
    "寝たらお腹が痛くなってきた",
    "【泣き声】",
    "泣き声",
    "【泣】",
    "泣",
    "【はじめの悲鳴】",
    "はじめの悲鳴",
    "お腹鳴った",
    "【はじめの鼻息】",
    "はじめの鼻息",
    "お腹空いた",
    "娘の泣き声",
    "お腹が痛い",
    "私もお腹が痛い",
    "お腹が痛そうだった",
    "【はじめの笑い声】",
    "はじめの笑い声",
    "【早送り】",
    "早送り",
    "【鳴き声】",
    "鳴き声",
    "お疲れ様です",
    "お疲れ様です。",
    "ごちそうさまでした",
    "ごちそうさまでした。",
    "お疲れ様でした",
    "お疲れ様",
    "おやすみ",
    "おやすみなさい",
    "サブタイトル&字幕 キョウ",
    "寝ますよ",
    "お腹すいた?",
    "お腹すいた",
    "1.5ページ",
    "2.5ページ",
    "3ページ",
    "4ページ",
    "5ページ",
    "♪",
    "★",
    "★おうくん、おもちゃを取ったよ★",
    "おうくん、おもちゃを取ったよ",
    "笑",
    "笑う?",
    "笑う",
    "COOK",
    "Tee",
    "3. 茹でた豚肉を茹でる",
    "4. 豚肉を茹でる",
    "5. 豚肉を茹でる",
    "6. 豚肉を茹でる",
    "7. 豚肉を茹でる",
    "8. 豚肉を茹でる",
    "9. 豚肉を茹でる",
    "ご覧いただきありがとうございます。良い1日を!",
    "ご覧いただきありがとうございます。良い1日を",
    "★ごきげんよう、また見てね★",
    "ごきげんよう、また見てね",
    "、",
    "、、",
    "、、、",
    "、、、、",
    "、、、、、",
    "、、、、、、",
    "、、、、、、、",
    "、、、、、、、、",
    "、、、、、、、、、",
    "、、、、、、、、、、",
    "、、、、、、、、、、、",
    "www",
    "wwwww",
    "🐕",
    "🐕🐕",
    "💦",
    "🐬",
    "🐾",
    "🐦",
    "📢",
    "📢📢",
    "📢📢📢",
    "📢📢📢📢",
    "📢📢📢📢📢",
    "📢📢📢📢📢📢",
    "📢📢📢📢📢📢📢",
    "📢📢📢📢📢📢📢📢",
    "📢📢📢📢📢📢📢📢📢",
    "📢📢📢📢📢📢📢📢📢📢",
    "📢📢📢📢📢📢📢📢📢📢📢",
    "📢📢📢📢📢📢📢📢📢📢📢📢",
    "📢📢📢📢📢📢📢📢📢📢📢📢📢",
    "pirate",
    "はじめしゃちょーエンディング",
    "【はじめしゃちょーエンディング】",
    "【はじめの泣き声】",
    "はじめの泣き声",
    "【赤ちゃんの泣き声】",
    "赤ちゃんの泣き声",
    "おーちゅー",
    "1",
    "【はじめの咳】",
    "はじめの咳",
    "【ED】",
    "ED",
    "パチ",
    "パチパチ",
    "パチパチパチ",
    "パチパチパチパチ",
    "パチパチパチパチパチ",
    "パチパチパチパチパチパチ",
    "パチパチパチパチパチパチパチ",
    "パチパチパチパチパチパチパチパチ",
    "3,2,1",
    "エクエク",
    "エクエクエク",
    "エクエクエクエク",
    "エクエクエクエクエク",
    "エクエクエクエクエクエク",
    "エクエクエクエクエクエクエク",
    "エクエクエクエクエクエクエクエク",
    "エクエクエクエクエクエクエクエクエク",
    "クックク",
    "クックック",
    "クッククック!",
    "クッククック",
    "クッククックク",
    "キチン",
    "★ごきげんよう、また見てね★",
    "ごきげんよう、また見てね",
    "ヘッ",
    "ヘッヘ",
    "ヘッヘッ",
    "ヘッヘッヘ",
    "ヘッヘッヘッ",
    "ヘッヘッヘッヘ",
    "ヘッヘッヘッヘッ",
    "ヘッヘッヘッヘッヘ",
    "ヘッヘッヘッヘッヘッ",
    "ヘッヘッヘッヘッヘッヘ",
    "ヘッヘッヘッヘッヘッヘッ",
    "ヘッヘッヘッヘッヘッヘッヘ",
    "ヘッヘッヘッヘッヘッヘッヘッ",
    "プル",
    "プルプル",
    "プルプル",
    "プルプルプル",
    "プルプルプルプル",
    "プルプルプルプルプル",
    "プルプルプルプルプルプル",
    "プルプルプルプルプルプルプル",
    "プルプルプルプルプルプルプルプル",
    "ウンッ",
    "ウンッウンッ",
    "ウンッウンッウンッ",
    "ウンッウンッウンッウンッ",
    "ウンッウンッウンッウンッウンッ",
    "ウンッウンッウンッウンッウンッウンッ",
    "ウンッウンッウンッウンッウンッウンッウンッ",
    "ウンッウンッウンッウンッウンッウンッウンッウンッ",
    "ウンッンッ",
    "ウンッンッンッ",
    "ウンッンッンッンッ",
    "ウンッンッンッンッンッ",
    "ウンッンッンッンッンッンッ",
    "ウンッンッンッンッンッンッンッ",
    "ウンッンッンッンッンッンッンッンッ",
    "アアアアアアアアア",
    "アアアアアアアア",
    "アアアアアアア",
    "アアアアアア",
    "アアアアア",
    "アアアア",
    "アアア",
    "アア",
    "ア",
    "アゥ!",
    "アゥ",
    "アアッ!",
    "アアッ",
    "きゃ…",
    "ああ…きゃ…",
    "ああ…きゃ",
    "あ…あ…きゃ…",
    "あ…あ…きゃ",
    "きゃ",
    "ああきゃ",
    "アッ!",
    "アッ!アッ!",
    "アッ!アッ!アッ!",
    "アッ!アッ!アッ!アッ!",
    "アッ!アッ!アッ!アッ!アッ!",
    "アッ!アッ!アッ!アッ!アッ!アッ!",
    "アッ!アッ!アッ!アッ!アッ!アッ!アッ!",
    "アッ!アッ!アッ!アッ!アッ!アッ!アッ!アッ!",
    "アッ!アッ!アッ!アッ!アッ!アッ!アッ!アッ!アッ!",
    "アッアッ",
    "アッアッアッ",
    "アッアッアッアッ",
    "アッアッアッアッアッ",
    "アッアッアッアッアッアッ",
    "アッアッアッアッアッアッアッ",
    "アッアッアッアッアッアッアッアッ",
    "アッアッアッアッアッアッアッアッアッ",
    "アッアッアッアッアッアッアッアッアッアッ",
    "アッアッアッアッアッアッアッアッアッアッアッ",
    "アッ",
    "アッハ",
    "アッハッ",
    "アッハッハ",
    "アッハッハッ",
    "アッハッハッハ",
    "アッハッハッハッ",
    "アッハッハッハッハ",
    "アッハッハッハッハッ",
    "アッハッハッハッハッハッ",
    "アッハッハッハッハッハッハッ",
    "アッハッハッハッハッハッハッハッ",
    "アァァァァァァァァ",
    "アァァァァァァァ",
    "アァァァァァァ",
    "アァァァァァ",
    "アァァァァ",
    "アァァァ",
    "アァァ",
    "アァ",
    "アァア",
    "アァアァ",
    "アァアァア",
    "アァアァアァ",
    "アァアァアァア",
    "アァアァアァアァ",
    "アァアァアァアァア",
    "アァアァアァアァアァ",
    "アァアァアァアァアァア",
    "アァアァアァアァアァアァ",
    "アァアァアァアァアァアァアァ",
    "アー",
    "アーッ",
    "アーアー",
    "アーアーアー",
    "アーアーアーアー",
    "アーアーアーアーアー",
    "アーアーアーアーアーアー",
    "アーアーアーアーアーアーアー",
    "アー",
    "アーー",
    "アーーー",
    "アーーーー",
    "アーーーーー",
    "アーーーーーー",
    "アーーーーーーー",
    "アーーーーーーーー",
    "アーアーー",
    "アーアーーー",
    "アーアーーーー",
    "アーアーーーーー",
    "アーアーーーーーー",
    "アーアーーーーーーー",
    "アーアーーーーーーーー",
    "アハッ!",
    "アハッ!アハッ!",
    "アハッ!アハッ!アハッ!",
    "アハッ!アハッ!アハッ!アハッ!",
    "アハッ!アハッ!アハッ!アハッ!アハッ!",
    "アハッ!アハッ!アハッ!アハッ!アハッ!アハッ!",
    "アハッ!アハッ!アハッ!アハッ!アハッ!アハッ!アハッ!",
    "アハッ!アハッ!アハッ!アハッ!アハッ!アハッ!アハッ!アハッ!",
    "アハッ",
    "アハッアハッ!",
    "アハッアハッ",
    "アハッアハッアハッ",
    "アハッアハッアハッアハッ",
    "アハッアハッアハッアハッアハッ",
    "アハッアハッアハッアハッアハッアハッ",
    "アハッアハッアハッアハッアハッアハッアハッ",
    "アハッアハッアハッアハッアハッアハッアハッアハッ",
    "アハッアハッアハッアハッアハッアハッアハッアハッアハッ",
    "はい",
    "はあ",
    "はあは",
    "はあはあ",
    "はあはあは",
    "はあはあはあ",
    "はあはあはあは",
    "はあはあはあはあ",
    "はあはあはあはあは",
    "はあはあはあはあはあ",
    "はあはあはあはあはあは",
    "はあはあはあはあはあはあ",
    "はあはあはあはあはあはあは"
    "はあはあはあはあはあはあはあ",
    "はあはあはあはあはあはあはあは",
    "はあはあはあはあはあはあはあはあ",
    "あは",
    "あはあは",
    "あはあはあは",
    "あはあはあはあは",
    "あはあはあはあはあは",
    "あはあはあはあはあはあは",
    "あはあはあはあはあはあはあは",
    "あはうん",
    "ふふふ、あー、あー、あー、あー",
    "ふふふ、あー、あー、あー",
    "ふふふ、あー、あー",
    "ふふふ、あー",
    "ふふふ、あー、ふふふ、あー",
    "ふふふ、あー、ふふふ、あー、ふふふ、あー",
    "ふふふふふふふふふふ",
    "ふふふふふふふふふ",
    "ふふふふふふふふ",
    "ふふふふふふふ",
    "ふふふふふふ",
    "ふふふふふ",
    "ふふふふ",
    "ふふふ。",
    "ふふふ",
    "ふふ",
    "ふ",
    "む",
    "ふん",
    "ふんんん",
    "ふんんん",
    "ふんんんん",
    "ふんんんんん",
    "ふんんんんんん",
    "ふぅぅぅぅぅぅぅん",
    "ふぅぅぅぅぅぅん",
    "ふぅぅぅぅぅん",
    "ふぅぅぅぅん",
    "ふぅぅぅん",
    "ふぅぅん",
    "ふぅん",
    "ふぅんん",
    "ふぅんんん",
    "ふぅんんんん",
    "ふぅんんんんん",
    "ふぅんんんんんん",
    "ふぅぅんん",
    "ふぅぅんん",
    "ふぅぅんんん",
    "ふぅぅんんんん",
    "ふぅぅんんんんん",
    "ふぅぅんんんんんん",
    "ふぅぅぅんん",
    "ふぅぅぅんんん",
    "ふぅぅぅんんんん",
    "ふぅぅぅんんんんん",
    "ふぅぅぅんんんんんん",
    "ふー",
    "ふーー",
    "ふーーー",
    "ふーーーー",
    "ふーーーーー",
    "ふーーーーーー",
    "ふーーーーーーー",
    "ふーーーーーーーー",
    "ふーーーーーーーーー",
    "ふーふー",
    "ふーふーふー",
    "ふーふーふーふー",
    "ふーふーふーふーふー",
    "ふーふーふーふーふーふー",
    "ふーふーふーふーふーふーふーふー",
    "ふーふーふーふーふーふーふーふーふーふー",
    "ふぅ",
    "ふぅふぅ",
    "ふぅふぅふぅ",
    "ふぅふぅふぅふぅ",
    "ふぅふぅふぅふぅふぅ",
    "ふぅふぅふぅふぅふぅふぅ",
    "ふぅふぅふぅふぅふぅふぅふぅ",
    "ふぅふぅふぅふぅふぅふぅふぅふぅ",
    "おーん",
    "おう",
    "おう!",
    "おうう",
    "おううう",
    "おうううう",
    "おううううう",
    "おうううううう",
    "おううううううう",
    "おうううううううう",
    "おううううううううう",
    "おうう!",
    "おううう!",
    "おうううう!",
    "おううううう!",
    "おうううううう!",
    "おううううううう!",
    "おうううううううう!",
    "おううううううううう!",
    "かす",
    "お",
    "お、",
    "お、お",
    "お、お、",
    "お、お、お",
    "お、お、お、",
    "お、お、お、お",
    "お、お、お、お、",
    "お、お、お、お、お",
    "お、お、お、お、お、",
    "お、お、お、お、お、お",
    "お、お、お、お、お、お、",
    "お、お、お、お、お、お、お",
    "お、お、お、お、お、お、お、",
    "お、お、お、お、お、お、お、お",
    "お、お、お、お、お、お、お、お、",
    "お、お、お、お、お、お、お、お、お",
    "お、お、お、お、お、お、お、お、お、",
    "おおっ",
    "おおっっ",
    "おおっっっ",
    "おおっっっっ",
    "おおっっっっっ",
    "おおっっっっっっ",
    "おおっっっっっっっ",
    "おおっっっっっっっっ",
    "おおおっ",
    "おおおっっ",
    "おおおっっっ",
    "おおおっっっっ",
    "おおおっっっっっ",
    "おおおっっっっっっ",
    "おおおっっっっっっっ",
    "おおおっっっっっっっっ",
    "おおおおっ",
    "おおおおっっ",
    "おおおおっっっ",
    "おおおおっっっっ",
    "おおおおっっっっっ",
    "おおおおっっっっっっ",
    "おおおおっっっっっっっ",
    "おおおおっっっっっっっっ",
    "おおおおおっ",
    "おおおおおっっ",
    "おおおおおっっっ",
    "おおおおおっっっっ",
    "おおおおおっっっっっ",
    "おおおおおっっっっっっ",
    "おおおおおっっっっっっっ",
    "おおおおおっっっっっっっっ",
    "おおおおおおっ",
    "おおおおおおっっ",
    "おおおおおおっっっ",
    "おおおおおおっっっっ",
    "おおおおおおっっっっっ",
    "おおおおおおっっっっっっ",
    "おおおおおおっっっっっっっ",
    "おおおおおおっっっっっっっっ",
    "おおおおおおおっ",
    "おおおおおおおっっ",
    "おおおおおおおっっっ",
    "おおおおおおおっっっっ",
    "おおおおおおおっっっっっ",
    "おおおおおおおっっっっっっ",
    "おおおおおおおっっっっっっっ",
    "おおおおおおおっっっっっっっっ",
    "おおおおおおおっ",
    "おおおおおおおおっっ",
    "おおおおおおおおっっっ",
    "おおおおおおおおっっっっ",
    "おおおおおおおおっっっっっ",
    "おおおおおおおおっっっっっっ",
    "おおおおおおおおっっっっっっっ",
    "おおおおおおおおっっっっっっっっ",
    "おお",
    "おおお",
    "おおおお",
    "おおおおお",
    "おおおおおお",
    "おおおおおおお",
    "おおおおおおおお",
    "おおおおおおおおお",
    "お!",
    "おぉ",
    "あ・・・",
    "あ〜、う〜ん",
    "あ〜、う〜ん、う〜ん",
    "あ〜、う〜ん、う〜ん、う〜ん",
    "あ〜、う〜ん、う〜ん、う〜ん、う〜ん",
    "あ〜、う〜ん、う〜ん、う〜ん、う〜ん、う〜ん",
    "あ〜、う〜ん、う〜ん、う〜ん、う〜ん、う〜ん、う〜ん",
    "あ〜、う〜ん、う〜ん、う〜ん、う〜ん、う〜ん、う〜ん、う〜ん",
    "あ〜う〜ん",
    "あ〜う〜んう〜ん",
    "あ〜う〜んう〜んう〜ん",
    "あ〜う〜んう〜んう〜んう〜ん",
    "あ〜う〜んう〜んう〜んう〜んう〜ん",
    "あ〜う〜んう〜んう〜んう〜んう〜んう〜ん",
    "あ〜う〜んう〜んう〜んう〜んう〜んう〜んう〜ん",
    "あ〜う〜んう〜んう〜んう〜んう〜んう〜んう〜んう〜ん",
    "あ〜う〜んう〜んう〜んう〜んう〜んう〜んう〜んう〜んう〜ん",
    "あ〜",
    "あ〜〜",
    "あ〜〜〜",
    "あ〜〜〜〜",
    "あ〜〜〜〜〜",
    "あ〜〜〜〜〜〜",
    "あ〜〜〜〜〜〜〜",
    "あ〜〜〜〜〜〜〜〜",
    "あ〜〜〜〜〜〜〜〜〜",
    "あ〜〜〜〜〜〜〜〜〜〜",
    "あ〜〜ー",
    "あ〜〜ーー",
    "あ〜〜ーーー",
    "あ〜〜ーーーー",
    "あ〜〜ーーーーー",
    "あ〜〜ーーーーーー",
    "あ〜〜ーーーーーーー",
    "あ〜〜ーーーーーーーー",
    "あ〜〜ーーーーーーーーー",
    "あ〜〜ーーーーーーーーーー",
    "あ〜〜ーん",
    "あ〜〜ーんん",
    "あ〜〜ーんんん",
    "あ〜〜ーんんんん",
    "あ〜〜ーんんんんん",
    "あ〜〜ーんんんんんん",
    "あ〜〜ーんんんんんんん",
    "あ〜〜ーんんんんんんんん",
    "あ〜〜ーんんんんんんんんん",
    "あ〜〜ーんんんんんんんんんん",
    "あ〜あ〜",
    "あ〜あ〜あ〜",
    "あ〜あ〜あ〜あ〜",
    "あ〜あ〜あ〜あ〜あ〜",
    "あ〜あ〜あ〜あ〜あ〜あ〜",
    "あ〜あ〜あ〜あ〜あ〜あ〜あ〜",
    "あ〜あ〜あ〜あ〜あ〜あ〜あ〜あ〜",
    "あ〜あ〜あ〜あ〜あ〜あ〜あ〜あ〜あ〜",
    "あ〜あくっ",
    "あくっ",
    "あーすー",
    "あーすーあーすー",
    "あーすーあーすーあーすー",
    "あーすーあーすーあーすーあーすー",
    "あーすーあーー",
    "あーすーあーーー",
    "あーすーあーーーー",
    "あーすーあーーーーー",
    "あーすーあーーーーーー",
    "あーすーあーーーーーーー",
    "あーすーあーーーーーーーー",
    "あ〜ん",
    "あ〜んん",
    "あ〜んんん",
    "あ〜んんんん",
    "あ〜んんんんん",
    "あ〜んんんんんん",
    "あ〜んんんんんんん",
    "あ〜んんんんんんんん",
    "あゝぬぅあゝぬぅあゝぬぅあゝぬぅあゝぬぅあゝぬぅ",
    "あゝぬぅあゝぬぅあゝぬぅあゝぬぅあゝぬぅ",
    "あゝぬぅあゝぬぅあゝぬぅあゝぬぅ",
    "あゝぬぅあゝぬぅあゝぬぅ",
    "あゝぬぅあゝぬぅ",
    "あゝぬぅ",
    "あゝぬぅぬぅぬぅぬぅぬぅぬぅぬぅ",
    "あゝぬぅぬぅぬぅぬぅぬぅぬぅ",
    "あゝぬぅぬぅぬぅぬぅぬぅ",
    "あゝぬぅぬぅぬぅぬぅ",
    "あゝぬぅぬぅぬぅ",
    "あゝぬぅぬぅ",
    "あゝぬぅ",
    "あゝ",
    "あゝあゝ",
    "あゝあゝあゝ",
    "あゝあゝあゝあゝ",
    "あゝあゝあゝあゝあゝ",
    "あゝあゝあゝあゝあゝあゝ",
    "あゝあゝあゝあゝあゝあゝあゝ",
    "あゝあゝあゝあゝあゝあゝあゝあゝ",
    "ぬあ゛あ゛",
    "ぬあ゛あ",
    "ぬあ゛",
    "うつ",
    "うつつ",
    "うつつつ",
    "うつつつつ",
    "うつつつつつ",
    "うつつつつつつ",
    "うつつつつつつつ",
    "うつつつつつつつつ",
    "hand",
    "in",
    "う〜ん",
    "う〜んん",
    "う〜んんん",
    "う〜んんんん",
    "う〜んんんんん",
    "う〜んんんんんん",
    "う〜んんんんんんん",
    "う゛あ゛",
    "う゛あ゛あ゛",
    "う゛あ゛あ゛あ゛",
    "う゛あ゛あ゛あ゛あ゛",
    "う゛あ゛あ゛あ゛あ゛あ゛",
    "う゛あ゛あ゛あ゛あ゛あ゛あ",
    "う゛あ゛あ゛あ゛あ゛あ゛あ゛あ",
    "う゛あ゛あ゛あ゛あ゛あ゛あ゛あ゛あ",
    "う゛あ゛あ゛あ゛あ゛あ゛あ゛あ゛あ゛あ",
    "う゛あ゛あ゛あ゛あ゛あ゛あ゛あ゛あ゛あ゛あ",
    "えが",
    "うあ",
    "うああ",
    "うあああ",
    "うああああ",
    "うあああああ",
    "うああああああ",
    "うあああああああ",
    "うああああああああ",
    "うあああああああああ",
    "うああああああああああ",
    "うあああああああああああ",
    "うああああああああああああ",
    "うあああああああああああああ",
    "うああああああああああああああ",
    "あ~~あ~~あ~~あ~~あ~~あ~~あ~~あ~~あ~~あ~~あ~~あ~~",
    "あ~~あ~~あ~~あ~~あ~~あ~~あ~~あ~~あ~~あ~~あ~~",
    "あ~~あ~~あ~~あ~~あ~~あ~~あ~~あ~~あ~~あ~~",
    "あ~~あ~~あ~~あ~~あ~~あ~~あ~~あ~~あ~~",
    "あ~~あ~~あ~~あ~~あ~~あ~~あ~~あ~~",
    "あ~~あ~~あ~~あ~~あ~~あ~~あ~~",
    "あ~~あ~~あ~~あ~~あ~~あ~~",
    "あ~~あ~~あ~~あ~~あ~~",
    "あ~~あ~~あ~~あ~~",
    "あ~~あ~~あ~~",
    "あ~~あ~~",
    "あ~~",
    "あ~",
    "あ~~~",
    "あ~~~~",
    "あ~~~~~",
    "あ~~~~~~",
    "あ~~~~~~~",
    "あ~~ん",
    "あ~~んん",
    "あ~~んんん",
    "あ~~んんんん",
    "あ~~んんんんん",
    "あ~~んんんんんん",
    "あ~~んんんんんんん",
    "あ~~んんんんんんんん",
    "あ~~んあ~~ん",
    "あ~~んあ~~んあ~~ん",
    "あ~~んあ~~んあ~~んあ~~ん",
    "あ~~んあ~~んあ~~んあ~~んあ~~ん",
    "あ~~んあ~~んあ~~んあ~~んあ~~んあ~~ん",
    "あ~~んあ~~んあ~~んあ~~んあ~~んあ~~んあ~~ん",
    "あ~~んあ~~んあ~~んあ~~んあ~~んあ~~んあ~~んあ~~ん",
    "あ~~んあ~~んあ~~んあ~~んあ~~んあ~~んあ~~んあ~~んあ~~ん",
    "あ~~んあ~~んあ~~んあ~~んあ~~んあ~~んあ~~んあ~~んあ~~んあ~~ん",
    "あ~!",
    "ああ~",
    "ああ~~",
    "ああ~~~",
    "めめめ",
    "めめめめ",
    "めめめめめ",
    "めめめめめめ",
    "めめめめめめめ",
    "めめめめめめめめ",
    "めめめめめめめめめ",
    "めめめめめめめめめめ",
    "めめめめめめめめめめめ",
    "めめめめめめめめめめめめ",
    "めめめめめめめめめめめめめ",
    "めめめめめめめめめめめめめめ",
    "めめんぜんぽ",
    "あんないくそ",
    "くぅ",
    "くぅくぅ",
    "くぅくぅくぅ",
    "くぅくぅくぅくぅ",
    "くぅくぅくぅくぅくぅ",
    "くぅくぅくぅくぅくぅくぅ",
    "くぅくぅくぅくぅくぅくぅくぅ",
    "くぅくぅくぅくぅくぅくぅくぅくぅ",
    "くぅ…",
    "くぅくぅ…",
    "くぅくぅくぅ…",
    "くぅくぅくぅくぅ…",
    "くぅくぅくぅくぅくぅ…",
    "くぅくぅくぅくぅくぅくぅ…",
    "くぅくぅくぅくぅくぅくぅくぅ…",
    "くぅくぅくぅくぅくぅくぅくぅくぅ…",
    "フー",
    "フーフー",
    "フーフーフー",
    "フーフーフーフー",
    "フーフーフーフーフー",
    "フルルル!",
    "フルルル",
    "フルルルル",
    "フルルルルル",
    "フルルルルルル",
    "フルルルルルルル",
    "フルルルルルルルル",
    "フルルルルルルルルル",
    "しー",
    "しーしー",
    "しーしーしー",
    "しーしーしーしー",
    "しーしーしーしー",
    "しーしーしーしーしー",
    "しーしーしーしーしーしー",
    "しーしーしーしーしーしーしー",
    "しーしーしーしーしーしーしーしー",
    "しー、しー",
    "しー、しー、しー",
    "しー、しー、しー、しー",
    "しー、しー、しー、しー、しー",
    "しー、しー、しー、しー、しー、しー",
    "しー、しー、しー、しー、しー、しー、しー",
    "しー、しー、しー、しー、しー、しー、しー、しー",
    "しー、しー、しー、しー、しー、しー、しー、しー、しー",
    "しー、しー、しー、しー、しー、しー、しー、しー、しー、しー",
    "チッ",
    "チョキ!",
    "チョキ",
    "フフ",
    "フフフフ",
    "フフフフフフ",
    "フフフフフフフフ",
    "フフッ",
    "フフッフフッ",
    "フフッフフッフフッ",
    "フフッフフッフフッフフッ",
    "カリッ!",
    "カリッ",
    "カリッカリッ",
    "カリッカリッカリッ",
    "カリッカリッカリッカリッ",
    "カリッカリッカリッカリッカリッ",
    "カリッカリッカリッカリッカリッカリッ",
    "カリッカリッカリッカリッカリッカリッカリッ",
    "カリッカリッカリッカリッカリッカリッカリッカリッ",
    "ぷっ",
    "ぷっぷっ",
    "ぷっぷっぷっ",
    "ぷっぷっぷっぷっぷっ",
    "ぷっぷっぷっぷっぷっぷっ",
    "ぷっぷっぷっぷっぷっぷっぷっ",
    "ぷっぷっぷっぷっぷっぷっぷっぷっ",
    "ぷっぷっぷっぷっぷっぷっぷっぷっぷっ",
    "ぷっぷっぷっぷっぷっぷっぷっぷっぷっぷっ",
    "ぷぷ",
    "ぷぷぷ",
    "ぷぷぷぷ",
    "ぷぷぷぷぷ",
    "ぷぷぷぷぷぷ",
    "ぷぷぷぷぷぷぷ",
    "ぷぷぷぷぷぷぷぷ",
    "ぷぷぷぷぷぷぷぷぷ",
    "ぷぷぷぷぷぷぷぷぷぷ",
    "ぷぷぷぷぷぷぷぷぷぷぷ",
    "ぷぷぷぷぷぷぷぷぷぷぷぷ",
    "ぷぷぷぷぷぷぷぷぷぷぷぷぷ",
    "ぷぷぷぷぷぷぷぷぷぷぷぷぷぷ",
    "ぷっっ",
    "ぷっっっ",
    "ぷっっっっ",
    "ぷっっっっっ",
    "ぷっっっっっっ",
    "ぷっっっっっっっ",
    "ぷっっっっっっっっ",
    "ぷっっっっっっっっっ",
    "ぷっ~",
    "ぷっ~~",
    "ぷっ~~~",
    "ぷっ~~~~",
    "ぷっ~~~~~",
    "ぷっ~~~~~~",
    "ぷっ~~~~~~~",
    "ぷっ~~~~~~~~",
    "ぷっ~~~~~~~~~",
    "ぷっ、ぷっ",
    "ぷっ、ぷっ、ぷっ",
    "ぷっ、ぷっ、ぷっ、ぷっ",
    "ぷっ、ぷっ、ぷっ、ぷっ、ぷっ",
    "ぷっ、ぷっ、ぷっ、ぷっ、ぷっ、ぷっ",
    "ぷっ、ぷっ、ぷっ、ぷっ、ぷっ、ぷっ、ぷっ",
    "ぷっ、ぷっ、ぷっ、ぷっ、ぷっ、ぷっ、ぷっ、ぷっ",
    "ぷっ、ぷっ、ぷっ、ぷっ、ぷっ、ぷっ、ぷっ、ぷっ、ぷっ",
    "ぷっ、",
    "ぷっ、ぷっ、",
    "ぷっ、ぷっ、ぷっ、",
    "ぷっ、ぷっ、ぷっ、ぷっ、",
    "ぷっ、ぷっ、ぷっ、ぷっ、ぷっ、",
    "ぷっ、ぷっ、ぷっ、ぷっ、ぷっ、ぷっ、",
    "ぷっ、ぷっ、ぷっ、ぷっ、ぷっ、ぷっ、ぷっ、",
    "ぷっ、ぷっ、ぷっ、ぷっ、ぷっ、ぷっ、ぷっ、ぷっ、",
    "ぷっ、ぷっ、ぷっ、ぷっ、ぷっ、ぷっ、ぷっ、ぷっ、ぷっ、",
    "あっぷぅんぅんぅんぅんぅんぅん",
    "あっぷぅんぅんぅんぅんぅん",
    "あっぷぅんぅんぅんぅん",
    "あっぷぅんぅんぅん",
    "あっぷぅんぅん",
    "あっぷぅん",
    "あっぷぅんんんんんんん",
    "あっぷぅんんんんんん",
    "あっぷぅんんんんん",
    "あっぷぅんんんん",
    "あっぷぅんんん",
    "あっぷぅんん",
    "あっぷぅん",
    "あっぷぅんあっぷぅん",
    "あっぷぅんあっぷぅんあっぷぅん",
    "あっぷぅんあっぷぅんあっぷぅんあっぷぅん",
    "あっぷぅんあっぷぅんあっぷぅんあっぷぅんあっぷぅん",
    "あっはぁ",
    "あっはっは",
    "あっはっはっは",
    "あっはっはっはっは",
    "あっはっはっはっはっは",
    "あっはっはっはっはっはっは",
    "はっはっはー",
    "はっはっは",
    "おしおしの",
    "おしおしのおしおしの",
    "んんんんんんんんんんんー",
    "んんんんんんんんんんー",
    "んんんんんんんんんー",
    "んんんんんんんんー",
    "んんんんんんんー",
    "んんんんんんー",
    "んんんんんー",
    "んんんんー",
    "んんんんー",
    "んんんー",
    "んんー",
    "んんんんんんんんんんん",
    "んんんんんんんんんん",
    "んんんんんんんんん",
    "んんんんんんんん",
    "んんんんんんん",
    "んんんんんん",
    "んんんんん",
    "んんんん",
    "んんんん",
    "んんん",
    "んん",
    "ん?",
    "んっはっはっはっは…",
    "んっはっ",
    "んっはっは",
    "んっはっはっ",
    "んっはっはっは",
    "んっはっはっはっ",
    "んっはっはっはっは",
    "んっはっはっはっはっ",
    "んっはっはっはっはっは",
    "んっはっはっはっはっはっ",
    "んっはっはっはっはっはっは",
    "んっはっはっはっはっはっはっ",
    "んっはっはっはっはっはっはっは",
    "んっはっはっはっはっはっはっはっ",
    "んんっ",
    "んんっっ",
    "んんっっっ",
    "んんっっっっ",
    "んんっっっっっ",
    "んんっっっっっっ",
    "んんっっっっっっっ",
    "んんっっっっっっっっ",
    "んんっっっっっっっっっ",
    "んんんっ",
    "んんんっっ",
    "んんんっっっ",
    "んんんっっっっ",
    "んんんっっっっっ",
    "んんんっっっっっっ",
    "んんんっっっっっっっ",
    "んんんっっっっっっっっ",
    "んんんっっっっっっっっっ",
    "んんんんっ",
    "んんんんっっ",
    "んんんんっっっ",
    "んんんんっっっっ",
    "んんんんっっっっっ",
    "んんんんっっっっっっ",
    "んんんんっっっっっっっ",
    "んんんんっっっっっっっっ",
    "んんんんっっっっっっっっっ",
    "んんんんんっ",
    "んんんんんっっ",
    "んんんんんっっっ",
    "んんんんんっっっっ",
    "んんんんんっっっっっ",
    "んんんんんっっっっっっ",
    "んんんんんっっっっっっっ",
    "んんんんんっっっっっっっっ",
    "んんんんんっっっっっっっっっ",
    "んんんんんんっ",
    "んんんんんんっっ",
    "んんんんんんっっっ",
    "んんんんんんっっっっ",
    "んんんんんんっっっっっ",
    "んんんんんんっっっっっっ",
    "んんんんんんっっっっっっっ",
    "んんんんんんっっっっっっっっ",
    "んんんんんんっっっっっっっっっ",
    "んんんんんんんっ",
    "んんんんんんんっっ",
    "んんんんんんんっっっ",
    "んんんんんんんっっっっ",
    "んんんんんんんっっっっっ",
    "んんんんんんんっっっっっっ",
    "んんんんんんんっっっっっっっ",
    "んんんんんんんっっっっっっっっ",
    "んんんんんんんっっっっっっっっっ",
    "あ〜、"
    "ん〜、あ〜",
    "ん〜、あ〜、あ〜",
    "ん〜、あ〜、あ〜、あ〜",
    "ん〜、あ〜、あ〜、あ〜、あ〜",
    "ん〜、あ〜、あ〜、あ〜、あ〜、あ〜",
    "ん〜、あ〜、あ〜、あ〜、あ〜、あ〜、あ〜",
    "ん〜、あ〜、ん〜、あ〜",
    "ん〜、あ〜、ん〜、あ〜、ん〜、あ〜",
    "ん〜、あ〜、ん〜、あ〜、ん〜、あ〜、ん〜、あ〜",
    "ん〜、あ〜、ん〜、あ〜、ん〜、あ〜、ん〜、あ〜、ん〜、あ〜",
    "ん〜、あ〜、ん〜、あ〜、ん〜、あ〜、ん〜、あ〜、ん〜、あ〜、ん〜、あ〜",
    "ウゥ",
    "ウゥゥ",
    "ウゥゥゥ",
    "ウゥゥゥゥ",
    "ウゥゥゥゥゥ",
    "ウゥゥゥゥゥゥ",
    "ウゥゥゥゥゥゥゥ",
    "ウゥゥゥゥゥゥゥゥ",
    "ウゥゥゥゥゥゥゥゥゥ",
    "ウゥゥゥゥゥゥゥゥゥゥ",
    "ウゥゥゥゥゥゥゥゥゥゥゥ",
    "ウゥゥゥゥゥゥゥゥゥゥゥゥ",
    "ん〜",
    "ん〜ん〜",
    "ん〜ん〜ん〜",
    "ん〜ん〜ん〜ん〜",
    "ん〜ん〜ん〜ん〜ん〜",
    "ん〜ん〜ん〜ん〜ん〜ん〜",
    "ん〜ん〜ん〜ん〜ん〜ん〜ん〜",
    "ん〜ん〜ん〜ん〜ん〜ん〜ん〜ん〜",
    "ん〜、ん〜、ん〜、、ん〜、、ん〜、、ん〜、、ん〜、、ん〜、、ん〜、"
    "ん〜、ん〜、ん〜、、ん〜、、ん〜、、ん〜、、ん〜、、ん〜、"
    "ん〜、ん〜、ん〜、、ん〜、、ん〜、、ん〜、、ん〜、"
    "ん〜、ん〜、ん〜、、ん〜、、ん〜、、ん〜、"
    "ん〜、ん〜、ん〜、、ん〜、、ん〜、"
    "ん〜、ん〜、ん〜、、ん〜、"
    "ん〜、ん〜、ん〜、"
    "ん〜、ん〜、"
    "ん〜、",
    "ん〜、ん〜",
    "ん〜、ん〜、ん〜",
    "ん〜、ん〜、ん〜、ん〜",
    "ん〜、ん〜、ん〜、ん〜、ん〜",
    "ん〜、ん〜、ん〜、ん〜、ん〜、ん〜",
    "ん〜〜",
    "ん〜〜〜",
    "ん〜〜〜〜",
    "ん〜〜〜〜〜",
    "ん〜〜〜〜〜〜",
    "ん〜〜〜〜〜〜〜",
    "ん〜〜〜〜〜〜〜〜",
    "ん〜〜〜〜〜〜〜〜〜",
    "ん",
    "んああ",
    "んあああ",
    "んああああ",
    "んあああああ",
    "んああああああ",
    "んあああああああ",
    "んああああああああ",
    "んあああああああああ",
    "ん、ああ",
    "んああ",
    "えー",
    "えーー",
    "えーーー",
    "えーーーー",
    "えーーーーー",
    "えーーーーーー",
    "えーーーーーーー",
    "えーーーーーーーー",
    "えーーーーーーーーー",
    "えーーーーーーーーーー",
    "えーーーーーーーーーーー",
    "あ!あく…あくに…あら…あら…あ!あ!あくの…あくまで…あ!",
    "あ!あく、あくに、あら、あら、あ、あ、あくの、あくの、あくまで、あ!",
    "ああく、あくに、あら、あら、あ、あ、あくの、あくの、あくまで、あ",
    "ああくあくにあらあらあああくのあくのあくまであ",
    "ああくあくにあらあらあああくのあくまであ",
    "えへへ",
    "えへへへ",
    "えへへへへ",
    "えへへへへへ",
    "えへへへへへへ",
    "えへへへへへへへ",
    "えへへへへへへへへ",
    "えへへへへへへへへへ",
    "えへへへへへへへへへへ",
    "ええええええ",
    "えええええええ",
    # "えええええ",
    # "ええええ",
    # "えええ",
    # "ええ",
    # "え",
    # "ええええええ?",
    # "えええええ?",
    # "ええええ?",
    # "えええ?",
    # "ええ?",
    # "え?",
    # "あの",
    # "あのー",
    "え〜",
    "え〜〜",
    "え〜〜〜",
    "え〜〜〜〜",
    "え〜〜〜〜〜",
    "え〜〜〜〜〜〜",
    "え〜〜〜〜〜〜〜",
    "え〜〜〜〜〜〜〜〜",
    "えぇ…",
    "えぇ",
    "えぇっ",
    "えぇっえぇっ",
    "えぇっえぇっえぇっ",
    "えぇっえぇっえぇっえぇっ",
    "えぇっえぇっえぇっえぇっえぇっ",
    "えぇっえぇっえぇっえぇっえぇっえぇっ",
    "えぇっっ",
    "えぇっっっ",
    "えぇっっっっ",
    "えぇっっっっっ",
    "えぇっっっっっっ",
    "えぇっっっっっっっ",
    "えぇっっっっっっっっ",
    "あ、あ、あわ、あい、あい",
    "あああわあいあい",
    "あいあい",
    "えぇー",
    "えぇーー",
    "えぇーーー",
    "えぇーーーー",
    "えぇーーーーー",
    "えぇーーーーーー",
    "えぇーーーーーーー",
    "えぇーーーーーーーー",
    "えぇーーーーーーーーー",
    "えぇーーーーーーーーーー",
    "えぇーーーーーーーーーーー",
    "えっへぇっ",
    "えっへぇっえっへぇっ",
    "えっへぇっえっへぇっえっへぇっ",
    "えっへぇっぇっ",
    "えっへぇっぇっぇっ",
    "えっへぇっぇっぇっぇっ",
    "えっへぇっぇっぇっぇっぇっ",
    "えっへぇっっ",
    "えっへぇっっっ",
    "えっへぇっっっっ",
    "えっへぇっっっっっ",
    "えっへぇっっっっっっ",
    "えっへぇっ!",
    "えっへぇっえっへぇっ!",
    "えっへぇっえっへぇっえっへぇっ!",
    "えっへぇっぇっ!",
    "えっへぇっぇっぇっ!",
    "えっへぇっぇっぇっぇっ!",
    "えっへぇっぇっぇっぇっぇっ!",
    "えっへぇっっ!",
    "えっへぇっっっ!",
    "えっへぇっっっっ!",
    "えっへぇっっっっっ!",
    "えっへぇっっっっっっ!",
    "ん…",
    "んん…",
    "んんん…",
    "んんんん…",
    "んんんんんん…",
    "んんんんんんん…",
    "んんんんんんんん…",
    "んんんんんんんんん…",
    "んんんんんんんんんん…",
    "っ",
    "いひひ",
    "いひひひ",
    "いひひひひ",
    "いひひひひひ",
    "いひひひひひひ",
    "いひひひひひひひ",
    "いひひひひひひひひ",
    "いひひひひひひひひひ",
    "いひひひひひひひひひひ",
    "んんっ",
    "んんっ!",
    "んんっ!あ!",
    "んんっ!あ!あ",
    "んんっ!あ!あ!",
    "んんっ!あ!あ!あ!",
    "んんっ!あ!あ!あ!あ!",
    "んんっ!あ!あ!あ!あ!あ!",
    "んんっ!あ!あ!あ!あ!あ!あ!",
    "んんっ!あ!あ!あ!あ!あ!あ!あ!",
    "んんっ!あ!あ!あ!あ!あ!あ!あ!あ!",
    "んんっあ",
    "んんっああ",
    "んんっあああ",
    "んんっああああ",
    "んんっあああああ",
    "んんっああああああ",
    "んんっあああああああ",
    "んんっああああああああ",
    "んんっあああああああああ",
    "んーっ",
    "んーっっ",
    "んーっっっ",
    "んーっっっっ",
    "んーっっっっっ",
    "んーっっっっっっ",
    "んーっっっっっっっ",
    "んーっっっっっっっっ",
    "んーっっっっっっっっっ",
    "んーっんーっ",
    "んーっんーっんーっ",
    "んーっんーっんーっんーっ",
    "んーっんーっんーっんーっんーっ",
    "んーっんーっんーっんーっんーっんーっ",
    "んーっんーっんーっんーっんーっんーっんーっ",
    "んーっんーっんーっんーっんーっんーっんーっんーっ",
    "んーっんーっんーっんーっんーっんーっんーっんーっんーっ",
    "んー",
    "んー、",
    "んー、んー",
    "んー、んー、んー",
    "んー、んー、んー、んー",
    "んー、んー、んー、んー、んー",
    "んー、んー、んー、んー、んー、んー",
    "はぁ、はぁ、はぁ、はぁ、はぁ、はぁ、はぁ、はぁ、",
    "はぁ、はぁ、はぁ、はぁ、はぁ、はぁ、はぁ、",
    "はぁ、はぁ、はぁ、はぁ、はぁ、はぁ、",
    "はぁ、はぁ、はぁ、はぁ、はぁ、",
    "はぁ、はぁ、はぁ、はぁ、",
    "はぁ、はぁ、はぁ、",
    "はぁ、はぁ、",
    "はぁ、",
    "はぁ、はぁ",
    "はぁ、はぁ、はぁ",
    "はぁ、はぁ、はぁ、はぁ",
    "はぁ、はぁ、はぁ、はぁ、はぁ",
    "はぁ、はぁ、はぁ、はぁ、はぁ、はぁ",
    "はぁ、はぁ、はぁ、はぁ、はぁ、はぁ、はぁ",
    "はぁ、はぁ、はぁ、はぁ、はぁ、はぁ、はぁ、はぁ",
    "はぁ・・・",
    "はぁ",
    "はぁ〜",
    "はぁはぁ",
    "はぁはぁはぁ",
    "はぁはぁはぁはぁ",
    "はぁはぁはぁはぁはぁ",
    "はぁはぁはぁはぁはぁはぁ",
    "はぁはぁはぁはぁはぁはぁはぁ",
    "はぁはぁはぁはぁはぁはぁはぁはぁ",
    "はぁはぁはぁはぁはぁはぁはぁはぁはぁ",
    "はぁあぁ",
    "はぁあぁぁ",
    "はぁあぁぁぁ",
    "はぁあぁぁぁぁ",
    "はぁあぁぁぁぁぁ",
    "はぁあぁぁぁぁぁぁ",
    "はぁあぁぁぁぁぁぁぁ",
    "はぁあぁぁぁぁぁぁぁぁ",
    "はぁあぁぁぁぁぁぁぁぁぁ",
    "はぁあぁぁぁぁぁぁぁぁぁぁ",
    "はぁあぁあぁ",
    "はぁあぁあぁあぁ",
    "はぁあぁあぁあぁあぁ",
    "はぁあぁあぁあぁあぁあぁ",
    "はぁあぁあぁあぁあぁあぁあぁ",
    "はぁあぁあぁあぁあぁあぁあぁあぁ",
    "はぁあぁあぁあぁあぁあぁあぁあぁあぁ",
    "はぁあぁあぁあぁあぁあぁあぁあぁあぁあぁ",
    "はぁあぁぁあぁ",
    "はぁあぁぁあぁあぁ",
    "はぁあぁぁあぁぁ",
    "はぁあぁぁあぁあぁぁ",
    "はぁ…あぁ…",
    "はぁ…",
    "はぁ~",
    "はぁ~~",
    "はぁ~~~",
    "はぁ~~~~",
    "はぁ~~~~~",
    "はぁ~~~~~~",
    "はぁ~~~~~~~",
    "はぁ~~~~~~~~",
    "はぁぁ~",
    "はぁぁ~~",
    "はぁぁ~~~",
    "はぁはぁ~",
    "はぁはぁ~~",
    "はぁはぁ~~~",
    "あ゛あ゛あ",
    "おぉ",
    "あんっ",
    "あんっあんっ",
    "あんっあんっあんっ",
    "あんっあんっあんっあんっ",
    "あんっあんっあんっあんっあんっ",
    "あんっあんっあんっあんっあんっあんっ",
    "あんっあんっあんっあんっあんっあんっあんっ",
    "あんっっ",
    "あんっっっ",
    "あんっっっっ",
    "あんっっっっっ",
    "あんっっっっっっ",
    "あんっんっ",
    "あんっんっんっ",
    "あんっんっんっんっ",
    "あんっんっんっんっんっ",
    "あんっんっんっんっんっんっ",
    "あんっんっんっんっんっんっんっ",
    "んんんっ",
    "んんっ?",
    "んっ",
    "んっんっ",
    "んっんっんっ",
    "んっんっんっんっ",
    "んっんっんっんっんっ",
    "んっんっんっんっんっんっ",
    "んっんっんっんっんっんっんっ",
    "んっ、んっ、んっ、んっ、んっ、んっ、んっ、んっ",
    "んっ、んっ、んっ、んっ、んっ、んっ、んっ",
    "んっ、んっ、んっ、んっ、んっ、んっ",
    "んっ、んっ、んっ、んっ、んっ",
    "んっ、んっ、んっ、んっ",
    "んっ、んっ、んっ",
    "んっ、んっ",
    "んっ、",
    "んっんっ、",
    "んっんっんっ、",
    "んっんっんっんっ、",
    "んっんっんっんっんっ、",
    "んっんっんっんっんっんっ、",
    "んっんっんっんっんっんっんっ、",
    "んっ、んーーーーーー",
    "んっ、んーーーーー",
    "んっ、んーーーー",
    "んっ、んーーー",
    "んっ、んーー",
    "んっ、んー",
    "んっ、んー、",
    "んっ、んー、んー、",
    "んっ、んー、んー、んー、",
    "んっ、んー、んー、んー、んー、",
    "んっ、んー、んー、んー、んー、んー、",
    "んっ、んー、んー、んー、んー、んー、んー、",
    "んっ、んー、んー、んー、んー、んー、んー、んー、",
    "んっ、んー、んー。",
    "んっ、んー、んー、んー。",
    "んっ、んー、んー、んー、んー。",
    "んっ、んー、んー、んー、んー、んー。",
    "んっ、んー、んー、んー、んー、んー、んー。",
    "んっ、んー、んー、んー、んー、んー、んー、んー。",
    "んっ、んー、んー",
    "んっ、んー、んー、んー",
    "んっ、んー、んー、んー、んー",
    "んっ、んー、んー、んー、んー、んー",
    "んっ、んー、んー、んー、んー、んー、んー",
    "んっ、んー、んー、んー、んー、んー、んー、んー",
    "んーんーんーんーんーんーんー",
    "んーんーんーんーんーんー",
    "んーんーんーんーんー",
    "んーんーんーんー",
    "んーんーんー",
    "んーんー",
    "んーん",
    "んーんん",
    "んーんんん",
    "んーんんんん",
    "んーんんんんん",
    "んーんんんんんん",
    "んーんんんんんんん",
    "んーんんんんんんんん",
    "んーんんんんんんんんん",
    "んーんんんんんんんんんん",
    "んーんんんんんんんんんんん",
    "んーんんんんんんんんんんんん",
    "んーんんんんんんんんんんんんん",
    "んーんんんんんんんんんんんんんん",
    "んーんんんんんんんんんんんんんんん",
    "んーんんんんんんんんんんんんんんんん",
    "んーんんんんんんんんんんんんんんんんん",
    "んーんんんんんんんんんんんんんんんんんん",
    "んーんんんんんんんんんんんんんんんんんんん",
    "んーんんんんんんんんんんんんんんんんんんんん",
    "んーんんんんんんんんんんんんんんんんんんんんん",
    "んーんんんんんんんんんんんんんんんんんんんんんん",
    "んーんんんんんんんんんんんんんんんんんんんんんんん",
    "んーんんんんんんんんんんんんんんんんんんんんんんんん",
    "んーんんんんんんんんんんんんんんんんんんんんんんんんん",
    "んーんんんんんんんんんんんんんんんんんんんんんんんんんん",
    "んーんんんんんんんんんんんんんんんんんんんんんんんんんんん",
    "んーんんんんんんんんんんんんんんんんんんんんんんんんんんんん",
    "んーんんんんんんんんんんんんんんんんんんんんんんんんんんんんん",
    "んーんんんんんんんんんんんんんんんんんんんんんんんんんんんんんん",
    "んーんんんんんんんんんんんんんんんんんんんんんんんんんんんんんんん",
    "んーんんんんんんんんんんんんんんんんんんんんんんんんんんんんんんんん",
    "んーんんんんんんんんんんんんんんんんんんんんんんんんんんんんんんんんん",
    "んー、んー、んー、んー、んー、んー、んー",
    "んー、んー、んー、んー、んー、んー",
    "んー、んー、んー、んー、んー",
    "んー、んー、んー、んー",
    "んー、んー、んー",
    "んー、んー",
    "んー",
    "んー、んー、んー、んー、んー、んー、んー、",
    "んー、んー、んー、んー、んー、んー、",
    "んー、んー、んー、んー、んー、",
    "んー、んー、んー、んー、",
    "んー、んー、んー、",
    "んー、んー、",
    "んー、",
    "んー。",
    "んーんー。",
    "んーんーんー。",
    "んーんーんーんー。",
    "んーんーんーんーんー。",
    "んーんーんーんーんーんー。",
    "んーんーんーんーんーんーんー。",
    "ハッハッハッハッ",
    "ハッハッハッ",
    "ハッハッ",
    "ハッ",
    "ハハハッ",
    "ハハハッッ",
    "ハハハッッッ",
    "ハハハッッッッ",
    "ハハハッッッッッ",
    "ハハハッッッッッッ",
    "ハハハッッッッッッッ",
    "ハハハッッッッッッッッ",
    "ハハハッッッッッッッッッ",
    "ハハハッハハハッ",
    "ハハハッハハハッハハハッ",
    "ハハハッハハハッハハハッハハハッ",
    "ハハハッハハハッハハハッハハハッハハハッ",
    "ハハハッハハハッハハハッハハハッハハハッハハハッ",
    "ハハハッハハハッハハハッハハハッハハハッハハハッハハハッ",
    "ハハハッハハハッハハハッハハハッハハハッハハハッハハハッハハハッ",
    "ハハハハハハハハハハハッ",
    "ハハハハハハハハハハッ",
    "ハハハハハハハハハッ",
    "ハハハハハハハハ",
    "ハハハハハハハ",
    "ハハハハハハ",
    "ハハハハハ",
    "ハハハハ",
    "ハハハ",
    "ハハ",
    "ハ",
    "ハァッ",
    "おじいちゃんがおじいちゃんを救ってくれました。",
    "おじいちゃんがおじいちゃんを救ってくれました",
    "うむうむ",
    "あ、うーん",
    "あ、うーん、",
    "あ、うーん、うーん",
    "あ、うーん、うーん、",
    "ああ、うーん",
    "ああ、",
    "ああ、うーん、",
    "ああ、うーん、うーん",
    "ああ、うーん、うーん、",
    "あ、あ",
    "あ、あ、あ",
    "あ、あ、あ、あ",
    "あ、あ、あ、あ、あ",
    "あ、あ、あ、あ、あ、あ",
    "あ、あ、あ、あ、あ、あ、あ",
    "あ、あ、あ、あ、あ、あ、あ、あ",
    "あ、あ、あ、あ、あ、あ、あ、あ!",
    "あ、あ、あ、あ、あ、おーっとー",
    "あ、あ、あ、ん、あ、ん",
    "あ、あ、あ、ん、あ、ん、あ、ん",
    "あ、あ、あ、ん、あ、ん、あ、ん、あ、ん",
    "あ、あ、あ、ん、あ、ん、あ、ん、あ、ん、あ、ん",
    "あ、あ、あ、ん、あ、ん、あ、ん、あ、ん、あ、ん、あ、ん",
    "あ、あ、あ、ん、あ、ん、あ、ん、あ、ん、あ、ん、あ、ん、あ、ん",
    "あ、あ、あ、ん、あ、ん、あ、ん、あ、ん、あ、ん、あ、ん、あ、ん、あ、ん",
    "あああんあん",
    "あああんあんあん",
    "あああんあんあんあん",
    "あああんあんあんあんあん",
    "あああんあんあんあんあんあん",
    "あああんあんあんあんあんあんあん",
    "あああんあんあんあんあんあんあんあん",
    "あああんあんあんあんあんあんあんあんあん",
    "あああんあんあんあんあんあんあんあんあんあん",
    "あああんあんあんあんあんあんあんあんあんあんあん",
    "あああああおーっとー",
    "ああああおーっとー",
    "あああおーっとー",
    "ああおーっとー",
    "あ、あ、あ、あ、あ、あ、あ!",
    "あ、あ、あ、あ、あ、あ!",
    "あ、あ、あ、あ、あ!",
    "あ、あ、あ、あ!",
    "あ、あ、あ!",
    "あ、あいっ、あ!",
    "あ、あいっ、あ",
    "ああいっあ",
    "ああああお",
    "ああああおお",
    "ああああおおお",
    "ああああおおおお",
    "ああああおおおおお",
    "ああああおおおおおお",
    "ああああおおおおおおお",
    "ああああおおおおおおおお",
    "ああああおおおおおおおおお",
    "あ、あ、あ、あ、お",
    "あ、あ、あ、あ、お、お",
    "あ、あ、あ、あ、お、お、お、お",
    "あ、あ、あ、あ、お、お、お、お、お",
    "あ、あ、あ、あ、お、お、お、お、お、お",
    "あ、あ、あ、あ、お、お、お、お、お、お、お",
    "あ、あ、あ、あ、お、お、お、お、お、お、お、お",
    "あ、え、え、え、え、え、え?",
    "あ、え、え、え、え、え、え",
    "あ、え、え、え、",
    "あええ",
    "あえええ",
    "あええええ",
    "あえええええ",
    "あええええええ",
    "あえええええええ",
    "あええええええええ",
    "あえええええええええ",
    "あええええええええええ",
    "あ、え",
    "あ、え、",
    "あ、え、え",
    "あ、え、え、",
    "あ、え、え、え",
    "あ、え、え、え、",
    "あ、え、え、え、え",
    "あ、え、え、え、え、",
    "あ、え、え、え、え、え",
    "あ、え、え、え、え、え、",
    "あ、え、え、え、え、え、え",
    "あ、え、え、え、え、え、え、",
    "あ、え、え、え、え、え、え、え",
    "あ、え、え、え、え、え、え、え、",
    "あ、え、え、え、え、え、え、え、え",
    "あ、え、え、え、え、え、え、え、え、",
    "あ、え、え、え、え、え、え、え、え、え",
    "あ、え、え、え、え、え、え、え、え、え、",
    "あ、あ、え、え、え、え、え、え、え、え、え、え、え、",
    "あ、あ、え、え、え、え、え、え、え、え、え、え、え",
    "あ、あ、え、え、え、え、え、え、え、え、え、え、",
    "あ、あ、え、え、え、え、え、え、え、え、え、え",
    "あ、あ、え、え、え、え、え、え、え、え、え、",
    "あ、あ、え、え、え、え、え、え、え、え、え",
    "あ、あ、え、え、え、え、え、え、え、え、",
    "あ、あ、え、え、え、え、え、え、え、え",
    "あ、あ、え、え、え、え、え、え、え、",
    "あ、あ、え、え、え、え、え、え、え",
    "あ、あ、え、え、え、え、え、え、",
    "あ、あ、え、え、え、え、え、え",
    "あ、あ、え、え、え、え、え、",
    "あ、あ、え、え、え、え、え",
    "あ、あ、え、え、え、え、",
    "あ、あ、え、え、え、え",
    "あ、あ、え、え、え、",
    "あ、あ、え、え、え",
    "あ、あ、え、え、",
    "あ、あ、え、え",
    "あ、あ、え、",
    "あ、あ、え",
    "あ、あ!",
    "あ!",
    "あ!あ!",
    "あ!あ!あ!",
    "あ!あ!あ!あ!",
    "あ!あ!あ!あ!あ!",
    "あ!あ!あ!あ!あ!あ!",
    "あ!あ!あ!あ!あ!あ!あ!",
    "あ!あ!あ!あ!あ!あ!あ!あ!",
    "は〜",
    "は〜〜",
    "は〜〜〜",
    "は〜〜〜〜",
    "は〜〜〜〜〜",
    "は〜〜〜〜〜〜",
    "は〜〜〜〜〜〜〜",
    "は〜〜〜〜〜〜〜〜",
    "は〜〜〜〜〜〜〜〜〜",
    "は〜",
    "は〜は〜",
    "は〜は〜は〜",
    "は〜は〜は〜は〜",
    "は〜は〜は〜は〜は〜",
    "は〜は〜は〜は〜は〜は〜",
    "は〜は〜は〜は〜は〜は〜は〜",
    "は〜は〜は〜は〜は〜は〜は〜は〜",
    "は〜は〜は〜は〜は〜は〜は〜は〜は〜",
    "はー",
    "はーはー",
    "はーはーはー",
    "はーはーはーはー",
    "はーはーはーはーはー",
    "はーはーはーはーはーはー",
    "はーはーはーはーはーはーはー",
    "はーはーはーはーはーはーはーはー",
    "はーはーはーはーはーはーはーはーはー",
    "うわ",
    "うわわ",
    "うわわわ",
    "うわわわわ",
    "うわわわわわ",
    "うわわわわわわ",
    "うわわわわわわわ",
    "うわわわわわわわわ",
    "うわわわわわわわわわ",
    "うわうわ",
    "うわうわうわ",
    "うわうわうわうわ",
    "うわうわうわうわうわ",
    "うわうわうわうわうわうわ",
    "うわうわうわうわうわうわうわ",
    "うわうわうわうわうわうわうわうわ",
    "うわうわうわうわうわうわうわうわうわ",
    "はーー",
    "はーーー",
    "はーーーー",
    "はーーーーー",
    "はーーーーーー",
    "はーーーーーーー",
    "はーーーーーーーー",
    "はーーーーーーーーー",
    "はーーーーーーーーーー",
    "はーーーーーーーーーーー",
    "はーーーーーーーーーーーー",
    "はー、",
    "はー、は、",
    "はー、はー、",
    "はー、は",
    "はー、はー",
    "はー、はー、はー",
    "はー、はー、はー、はー",
    "はー、はー、はー、はー、はー",
    "はー、はー、はー、はー、はー、はー",
    "はっはー",
    "はっはーー",
    "はっはーーー",
    "はっはーーーー",
    "はっはーーーーー",
    "はっはーーーーーー",
    "はっあん",
    "はっはっあん",
    "はっはっはっあん",
    "はっはっはっはっあん",
    "はっはっはっはっはっあん",
    "はっはっはっはっはっはっあん",
    "はっはっはっはっはっはっはっあん",
    "はっはっはっはっはっはっはっはっあん",
    "はっはっはっはっはっはっはっはっはっあん",
    "はっはっはっはっはっはっはっはっはっはっあん",
    "はっはっはっはっはっはっはっはっはっはっはっあん",
    "はっあんあん",
    "はっはっあんあん",
    "はっはっはっあんあん",
    "はっはっはっはっあんあん",
    "はっはっはっはっはっあんあん",
    "はっはっはっはっはっはっあんあん",
    "はっはっはっはっはっはっはっあんあん",
    "はっはっはっはっはっはっはっはっあんあん",
    "はっはっはっはっはっはっはっはっはっあんあん",
    "はっはっはっはっはっはっはっはっはっはっあんあん",
    "はっはっはっはっはっはっはっはっはっはっはっあんあん",
    "あんあん",
    "はっ",
    "はっは",
    "はっはっ"
    "はっはっは",
    "はっはっはっ",
    "はっはっはっは",
    "はっはっはっはっ",
    "はっはっはっはっは",
    "はっはっはっはっはっ",
    "はっはっはっはっはっは",
    "はっはっはっはっはっはっ",
    "はっはっはっはっはっはっは",
    "はっはっはっはっはっはっはっ",
    "はっはっはっはっはっはっはっは",
    "はっはっはっはっはっはっはっは!",
    "はっはっはっはっはっはっはっは!!",
    "はっはっはっはっはっはっはっは!!!",
    "はっはっはっはっはっはっはっはー",
    "え、はっはっはっはっはっはっ",
    "え、はっはっはっはっはっ",
    "え、はっはっはっはっ",
    "え、はっはっはっ",
    "え、はっはっ",
    "え、はっ",
    "はっはっはっはっはっはっはっはっ",
    "はっはっはっはっはっはっはっ",
    "はっはっはっはっはっはっ",
    "はっはっはっはっはっ",
    "はっはっはっはっ",
    "はっはっはっ",
    "はっはっ",
    "はっ",
    "おっと",
    "おっと、おっと",
    "おっと、おっと、おっと",
    "おっと、おっと、おっと、おっと",
    "おっと、おっと、おっと、おっと、おっと",
    "おっと、おっと、おっと、おっと、おっと、おっと",
    "おっと、おっと、おっと、おっと、おっと、おっと、おっと",
    "おっと、おっと、おっと、おっと、おっと、おっと、おっと、おっと",
    "おっと、おっと、おっと、おっと、おっと、おっと、おっと、おっと、おっと",
    "おっと、",
    "おっと、おっと、",
    "おっと、おっと、おっと、",
    "おっと、おっと、おっと、おっと、",
    "おっと、おっと、おっと、おっと、おっと、",
    "おっと、おっと、おっと、おっと、おっと、おっと、",
    "おっと、おっと、おっと、おっと、おっと、おっと、おっと、",
    "おっと、おっと、おっと、おっと、おっと、おっと、おっと、おっと、",
    "おっと、おっと、おっと、おっと、おっと、おっと、おっと、おっと、おっと、",
    "おっと、おっとして",
    "おっとおっとして",
    "おっとおっと",
    "おっとおっとおっと",
    "おっとおっとおっとおっと",
    "おっとおっとおっとおっとおっと",
    "おっとおっとおっとおっとおっとおっと",
    "おっとおっとおっとおっとおっとおっとおっと",
    "おっとおっとおっとおっとおっとおっとおっとおっと",
    "おっとおっとおっとおっとおっとおっとおっとおっとおっと",
    "おっとおっとおっとおっとおっとおっとおっとおっとおっとおっと",
    "おっはっ",
    "おっはっっ",
    "おっはっっっ",
    "おっはっっっっ",
    "おっはっっっっっ",
    "おっはっっっっっっ",
    "おっはっっっっっっっ",
    "おっはっっっっっっっっ",
    "おっはっっっっっっっっっ",
    "おっはっっっっっっっっっっ",
    "おっはっおっはっ",
    "おやつ",
    "おっ",
    "おっっ",
    "おっっっ",
    "おっっっっ",
    "おっっっっっ",
    "おっっっっっっ",
    "おっっっっっっっ",
    "おっっっっっっっっ",
    "おっおっ",
    "おっおっおっ",
    "おっおっおっおっ",
    "おっおっおっおっおっ",
    "おっおっおっおっおっおっ",
    "おっおっおっおっおっおっおっ",
    "おっおっおっおっおっおっおっおっ",
    "おっおっおっおっおっおっおっおっおっ",
    "おー",
    "おーっ",
    "おーっっ",
    "おーっっっ",
    "おーっっっっ",
    "おーっっっっっ",
    "おーっっっっっっ",
    "おーっっっっっっっ",
    "おーっっっっっっっっ",
    "おーっっっっっっっっっ",
    "おーっおーっ",
    "おーっおーっおーっ",
    "おーっおーっおーっおーっ",
    "おーっおーっおーっおーっおーっ",
    "おーっおーっおーっおーっおーっおーっ",
    "おーっおーっおーっおーっおーっおーっおーっ",
    "おーっおーっおーっおーっおーっおーっおーっおーっ",
    "おーっおーっおーっおーっおーっおーっおーっおーっおーっ",
    "おー、おー",
    "おー、おー、おー",
    "おー、おー、おー、おー",
    "おー、おー、おー、おー、おー",
    "おー、おー、おー、おー、おー、おー",
    "おー、おー、おー、おー、おー、おー、おー",
    "おー、おー、おー、おー、おー、おー、おー、おー",
    "おーおー",
    "おーおーおー",
    "おーおーおーおー",
    "おーおーおーおーおー",
    "おーおーおーおーおーおー",
    "おーおーおーおーおーおーおー",
    "おーおーおーおーおーおーおーおー",
    "おーおーおーおーおーおーおーおーおー",
    "おーおーおーおーおーおーおーおーおーおー",
    "おーおーおーおーおーおーおーおーおーおーおー",
    "おーおーおーおーおーおーおーおーおーおーおーおー",
    "あ、",
    "あ、あ、",
    "あ、あ、あ、",
    "あ、あ、あ、あ、",
    "あ、あ、あ、あ、あ、",
    "ハァ…ハァ…ハァ…ハァ…ハァ…ハァ…ハァ",
    "ハァ…ハァ…ハァ…ハァ…ハァ…ハァ",
    "ハァ…ハァ…ハァ…ハァ…ハァ",
    "ハァ…ハァ…ハァ…ハァ",
    "ハァ…ハァ…ハァ",
    "ハァ…ハァ",
    "ハァ",
    "ハァ…",
    "ハァー",
    "ハァ…ハァ…",
    "ハァ…ハァ…ハァ…",
    "ハァ…ハァ…ハァ…ハァ…",
    "ハァ…ハァ…ハァ…ハァ…ハァ…",
    "ハァ…ハァ…ハァ…ハァ…ハァ…ハァ…ハァ…",
    "ハァハァハァハァハァハァハァハァ",
    "ハァハァハァハァハァハァハァ",
    "ハァハァハァハァハァハァ",
    "ハァハァハァハァハァ",
    "ハァハァハァハァ",
    "ハァハァハァ",
    "ハァハァ",
    "ハァ",
    "ドゥンッドゥンッ",
    "ん あっはぁっはぁっはぁっ",
    "んあっはぁっはぁっはぁっ",
    "んふ",
    "んふふ",
    "んふふふ",
    "んふふふふ",
    "んふふふふふ",
    "んふふふふふふ",
    "んふふふふふふふ",
    "んふふふふふふふふ",
    "んふふふふふふふふふ",
    "んふんふ",
    "んふんふんふ",
    "んふんふんふんふ",
    "んふんふんふんふんふ",
    "んふんふんふんふんふんふ",
    "んふんふんふんふんふんふんふ",
    "んふんふんふんふんふんふんふんふ",
    "んふんふんふんふんふんふんふんふんふ",
    "ふんふん",
    "ふんふんふん",
    "ふんふんふんふん",
    "ふんふんふんふんふん",
    "ふんふんふんふんふんふん",
    "ふんふんふんふんふんふんふん",
    "ふんふんふんふんふんふんふんふん",
    "ふんふんふんふんふんふんふんふんふん",
    "ふんふんふんふんふんふんふんふんふんふん",
    "ふんふんふんふんふんふんふんふんふんふんふん",
    "んんー",
    "んんーー",
    "んんーーー",
    "んんーーーー",
    "んんーーーーー",
    "んんーーーーーー",
    "うんうんうんうんうんうん",
    "うんうんうんうんうん",
    "うんうんうんうん",
    "うんうんうん",
    "うんうん",
    "うん",
    "うん…",
    "うんうん…",
    "うんうんうん…",
    "うんうんうんうん…",
    "うんうんうんうんうん…",
    "うんうんうんうんうんうん…",
    "うっうっうっうっうっうっうっうっ",
    "うっうっうっうっうっうっうっ",
    "うっうっうっうっうっうっ",
    "うっうっうっうっうっ",
    "うっうっうっうっ",
    "うっうっうっ",
    "うっうっ",
    "うっ",
    "うっ…",
    "うっ…うっ…",
    "うっ…うっ…うっ…",
    "うっ…うっ…うっ…うっ…",
    "うっ…うっ…うっ…うっ…うっ…",
    "うっ…うっ…うっ…うっ…うっ…うっ…",
    "うっ…うっ…うっ…うっ…うっ…うっ…うっ…",
    "うっ…うっ…うっ…うっ…うっ…うっ…うっ…うっ…",
    "うっ…うっ",
    "うっ…うっ…うっ",
    "うっ…うっ…うっ…うっ",
    "うっ…うっ…うっ…うっ…うっ",
    "うっ…うっ…うっ…うっ…うっ…うっ",
    "うっ…うっ…うっ…うっ…うっ…うっ…うっ",
    "うっ…うっ…うっ…うっ…うっ…うっ…うっ…うっ",
    "ドゥンッ",
    "はっ!はっはっはっはっはっ!",
    "う、う、う、う、う!",
    "う、う、う、う!",
    "う、う、う!",
    "う、う!",
    "う!",
    "ね、あ、あ",
    "ね、あ、あ、あ",
    "ね、あ、あ、あ、あ",
    "ね、あ、あ、あ、あ、あ",
    "ね、あ、あ、あ、あ、あ、あ",
    "ね、あ、あ、あ、あ、あ、あ、あ",
    "ね、あ、あ、あ、あ、あ、あ、あ、あ",
    "ね、あ、あ、あ、あ、あ、あ、あ、あ、あ",
    "あ、あ、あ、あ、あ、あ、あ、あ",
    "あ、あ、あ、あ、あ、あ、あ、あ、",
    "あ、あ、あ、あ、あ、あ、あ、あ、あ",
    "あ、あ、あ、あ、あ、あ、あ、あ、あ、",
    "あ、あ、あ、あ、あ、あ、あ、あ、あ、あ",
    "あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、",
    "あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ",
    "あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、",
    "あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ",
    "あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、",
    "あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ",
    "あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、",
    "あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ",
    "あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、",
    "あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ",
    "あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、",
    "あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ",
    "あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、",
    "近びも",
    "ねああ",
    "ねあああ",
    "ねああああ",
    "ねあああああ",
    "ねああああああ",
    "ねあああああああ",
    "ねああああああああ",
    "ねあああああああああ",
    "ねああああああああああ",
    "う、う、う、う、う、う、う、う、う",
    "う、う、う、う、う、う、う、う",
    "う、う、う、う、う、う、う",
    "う、う、う、う、う、う",
    "う、う、う、う、う",
    "う、う、う、う",
    "う、う、う",
    "う、う",
    "う",
    "うう",
    "ううっ",
    "ううっっ",
    "ううっっっ",
    "ううっっっっ",
    "ううっっっっっ",
    "ううっっっっっっ",
    "ううっっっっっっっ",
    "ううっっっっっっっっ",
    "ううっっっっっっっっっ",
    "うううっ",
    "うううっっ",
    "うううっっっ",
    "うううっっっっ",
    "うううっっっっっ",
    "うううっっっっっっ",
    "うううっっっっっっっ",
    "うううっっっっっっっっ",
    "うううっっっっっっっっっ",
    "うううう",
    "ううううっ",
    "ううううっっ",
    "ううううっっっ",
    "ううううっっっっ",
    "ううううっっっっっ",
    "ううううっっっっっっ",
    "ううううっっっっっっっ",
    "ううううっっっっっっっっ",
    "ううううっっっっっっっっっ",
    "ううう",
    "うううう",
    "ううううう",
    "うううううう",
    "ううううううう",
    "うううううううう",
    "ううううううううう",
    "ああ、うううううううう",
    "ああ、ううううううう",
    "ああ、うううううう",
    "ああ、ううううう",
    "ああ、うううう",
    "ああ、ううう",
    "ああ、うう",
    "ああ、う",
    "ああうううううううう",
    "ああううううううう",
    "ああうううううう",
    "ああううううう",
    "ああうううう",
    "ああううう",
    "ああうう",
    "ああう",
    "ああ、うう、ああ、うう",
    "ああ、うう、ああ、うう、ああ、うう",
    "ああ、うう、ああ、うう、ああ、うう、ああ、うう",
    "ああ、うう、ああ、うう、ああ、うう、ああ、うう、ああ、うう",
    "ああ、うう、ああ、うう、ああ、うう、ああ、うう、ああ、うう、ああ、うう",
    "ああううああうう",
    "ああううああううああうう",
    "ああううああううああううああうう",
    "ああううああううああううああううああうう",
    "ああううああううああううああううああううああうう",
    "ああ、ああ",
    "ああ、ああ、ああ",
    "ああ、ああ、ああ、ああ",
    "ああ、ああ、ああ、ああ、ああ",
    "ああ、ああ、ああ、ああ、ああ、ああ",
    "ああ、ああ、ああ、ああ、ああ、ああ、ああ",
    "ああ、ああ、ああ、ああ、ああ、ああ、ああ、ああ",
    "あああああああああああっ",
    "ああああああああああっ",
    "あああああああああっ",
    "ああああああああっ",
    "あああああああっ",
    "ああああああっ",
    "あああああっ",
    "ああああっ",
    "あああっ",
    "ああっ",
    "あっ",
    "あっはぁっ",
    "あっはぁっはぁっ",
    "あっはぁっはぁっはぁっ",
    "あっはぁっはぁっはぁっはぁっ",
    "あっはぁっはぁっはぁっはぁっはぁっ",
    "あっはぁっはぁっはぁっはぁっはぁっはぁっ",
    "あっはぁっはぁっはぁっはぁっはぁっはぁっはぁっ",
    "あっはぁっはぁっはぁっはぁっはぁっはぁっはぁっはぁっ",
    "あっはぁっはぁっはぁっはぁっはぁっはぁっはぁっはぁっはぁっ",
    "あっはぁっはぁっはぁっはぁっはぁっはぁっはぁっはぁっはぁっはぁっ",
    "ああああああ あっはぁっ",
    "ああああああ あっはぁっはぁっ",
    "ああああああ あっはぁっはぁっはぁっ",
    "あああああああああああああああああ",
    "ああああああああああああああああ",
    "あああああああああああああああ",
    "ああああああああああああああ",
    "あああああああああああああ",
    "ああああああああああああ",
    "あああああああああああ",
    "ああああああああああ",
    "あああああああああ",
    "ああああああああ",
    "あああああああ",
    "ああああああ",
    "あああああ",
    "ああああ",
    "あああ",
    "ああ",
    "あ",
    "ああはっ",
    "あ、あはっ",
    "ああはっは",
    "ああはっはっは",
    "ああはっはっはっは",
    "ああはっはっはっはっは",
    "ああはっはっはっはっはっは",
    "ああはっはっはっはっはっはっは",
    "ああはっはっはっはっはっはっはっは",
    "ああはっはっはっはっはっはっはっはっは",
    "ああはっはっはっはっはっはっはっはっはっは",
    "あはっは",
    "あはっはっは",
    "あはっはっはっは",
    "あはっはっはっはっは",
    "あはっはっはっはっはっは",
    "あはっはっはっはっはっはっは",
    "あはっはっはっはっはっはっはっは",
    "あはっはっはっはっはっはっはっはっは",
    "あはっはっはっはっはっはっはっはっはっは",
    "ぴゅっ",
    "ぴゅっっ",
    "ぴゅっっっ",
    "ぴゅっっっっ",
    "ぴゅっっっっっ",
    "ぴゅっっっっっっ",
    "ぴゅっっっっっっっ",
    "ぴゅっっっっっっっっ",
    "ぴゅっっっっっっっっっ",
    "ぴゅっぴゅっ",
    "ぴゅっぴゅっぴゅっ",
    "ぴゅっぴゅっぴゅっぴゅっ",
    "ぴゅっぴゅっぴゅっぴゅっぴゅっ",
    "ぴゅっぴゅっぴゅっぴゅっぴゅっぴゅっ",
    "ぴゅっぴゅっぴゅっぴゅっぴゅっぴゅっぴゅっ",
    "ぴゅっぴゅっぴゅっぴゅっぴゅっぴゅっぴゅっぴゅっ",
    "ぴゅっぴゅっぴゅっぴゅっぴゅっぴゅっぴゅっぴゅっぴゅっ",
    "ぴゅっぴゅっぴゅっぴゅっぴゅっぴゅっぴゅっぴゅっぴゅっぴゅっ",
    "あわあわ",
    "あわあわあわ",
    "あわあわあわあわ",
    "あわあわあわあわあわ",
    "あわあわあわあわあわあわ",
    "あわあわあわあわあわあわあわ",
    "あわあわあわあわあわあわあわあわ",
    "あわあわあわあわあわあわあわあわあわ",
    "あわあわあわあわあわあわあわあわあわあわ",
    "あわあわあわあわあわあわあわあわあわあわあわ",
    "あぁーあぁーあぁーあぁーあぁーあぁーあぁーあぁー",
    "あぁーあぁーあぁーあぁーあぁーあぁーあぁー",
    "あぁーあぁーあぁーあぁーあぁーあぁー",
    "あぁーあぁーあぁーあぁーあぁー",
    "あぁーあぁーあぁーあぁー",
    "あぁーあぁーあぁー",
    "あぁーあぁー",
    "あぁーーーーーーーー",
    "あぁーーーーーーー",
    "あぁーーーーーー",
    "あぁーーーーー",
    "あぁーーーー",
    "あぁーーー",
    "あぁーー",
    "あぁー",
    "あぁっ",
    "あぁっあぁっ",
    "あぁっあぁっあぁっ",
    "あぁっあぁっあぁっあぁっ",
    "あぁっあぁっあぁっあぁっあぁっ",
    "あぁっあぁっあぁっあぁっあぁっあぁっ",
    "あぁっあぁっあぁっあぁっあぁっあぁっあぁっ",
    "あぁっ",
    "あぁ",
    "あぁ…",
    "あぁぁ…",
    "あぁぁぁ…",
    "あぁぁぁぁ…",
    "あぁぁぁぁぁ…",
    "あぁぁぁぁぁぁ…",
    "あぁぁ",
    "あぁぁぁ",
    "あぁぁぁぁ",
    "あぁぁぁぁぁ",
    "あぁぁぁぁぁぁ",
    "あぁぁぁぁぁぁぁ",
    "あぁぁぁぁぁぁぁぁ",
    "あぁぁぁぁぁぁぁぁぁ",
    "あぁぁぁぁぁぁぁぁぁぁ",
    "あぁぁぁぁぁぁぁぁぁぁぁ",
    "あぁぁぁぁぁぁぁぁぁぁぁぁ",
    "あぁぁぁぁぁぁぁぁぁぁぁぁぁ",
    "あぁぁぁぁぁぁぁぁぁぁぁぁぁぁ",
    "あぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁぁァぁぁぁぁぁぁぁぁぁぁぁん",
    "あぁあぁ",
    "あぁあぁあぁ",
    "あぁあぁあぁあぁ",
    "あぁあぁあぁあぁあぁ",
    "あぁあぁあぁあぁあぁあぁ",
    "あぁあぁ…",
    "あぁあぁあぁ…",
    "あぁあぁあぁあぁ…",
    "あぁあぁあぁあぁあぁ…",
    "あぁあぁあぁあぁあぁあぁ…",
    "あぁ…あぁ…",
    "あぁ…あぁ…あぁ…",
    "あぁ…あぁ…あぁ…あぁ…",
    "あぁ…あぁ…あぁ…あぁ…あぁ…",
    "あぁ…あぁ…あぁ…あぁ…あぁ…あぁ…",
    "あぁ…あぁ…あぁ…あぁ…あぁ…あぁ…あぁ…",
    "あぁ…あぁ…あぁ…あぁ…あぁ…あぁ…あぁ…あぁ…",
    "ん~~~~~~",
    "ん~~~~~",
    "ん~~~~",
    "ん~~~",
    "ん~~",
    "ん~",
    "し",
    "ンー",
    "ンーー",
    "ンーーー",
    "ンーーーー",
    "ンーーーーー",
    "ンーーーーーー",
    "ンーーーーーーー",
    "ンーーーーーーーー",
    "ンーーーーーーーーー",
    "ンー!",
    "ンーー!",
    "ンーーー!",
    "ンーーーー!",
    "ンーーーーー!",
    "ンーーーーーー!",
    "ンーーーーーーー!",
    "ンーーーーーーーー!",
    "ンーーーーーーーーー!",
    "ンー! シャー!",
    "ンー! シャー",
    "シッ",
    "シッシッ",
    "シッシッシッ",
    "シッシッシッシッ",
    "シッシッシッシッシッ",
    "シッシッシッシッシッシッ",
    "シッシッシッシッシッシッシッ",
    "シッシッシッシッシッシッシッシッ",
    "シッシッシッシッシッシッシッシッシッ",
    "シッシッシッシッシッシッシッシッシッシッ",
    "うわぁ…",
    "うわぁ",
    "わぁわぁわぁわぁ",
    "ンン",
    "ンン",
    "ンンン",
    "ンンンン",
    "ンンンンン",
    "ンンンンンン",
    "ンンンンンンン",
    "ンンンンンンンン",
    "ンンンンンンンンン",
    "ンンンンンンンンンン",
    "バンッ!",
    "バァッ!",
    "おーー",
    "おーーー",
    "おーーーー",
    "おーーーーー",
    "おーーーーーー",
    "おーーーーーーー",
    "おーーーーーーーー",
    "おーーーーーーーーー",
    "はぁっはぁっはぁっはぁっはぁっ",
    "はぁっはぁっはぁっはぁっ",
    "はぁっはぁっはぁっ",
    "はぁっはぁっ",
    "はぁっ",
    "はぁっ…",
    "はぁっはぁっはぁっはっ",
    "はぁっはぁっはぁっ",
    "はぁっはぁっ",
    "はぁっ、はぁっ",
    "はぁっ、はぁっ、はぁっ",
    "はぁっ、はぁっ、はぁっ、はっ",
    "はぁっ、はぁっ、はぁっ、はっ、てんまっ",
    "あはぁ",
    "あはぁい",
    "つっつっつっつっつっつっつっつっ",
    "つっつっつっつっつっつっつっ",
    "つっつっつっつっつっつっ",
    "つっつっつっつっつっ",
    "つっつっつっつっ",
    "つっつっつっ",
    "つっつっ",
    "つっ",
    "つっ…",
    "うぅ",
    "うぅ、うぅ",
    "うぅ、うぅ、うぅ",
    "うぅ、うぅ、うぅ、うぅ",
    "うぅ、うぅ、うぅ、うぅ、うぅ",
    "うぅ、うぅ、うぅ、うぅ、うぅ、うぅ",
    "うぅ、うぅ、うぅ、うぅ、うぅ、うぅ、うぅ",
    "うぅ、うぅ、うぅ、うぅ、うぅ、うぅ、うぅ、うぅ",
    "うぅ、うぅ、うぅ、うぅ、うぅ、うぅ、うぅ、うぅ、うぅ",
    "うぅ、",
    "うぅ、うぅ、",
    "うぅ、うぅ、うぅ、",
    "うぅ、うぅ、うぅ、うぅ、",
    "うぅ、うぅ、うぅ、うぅ、うぅ、",
    "うぅ、うぅ、うぅ、うぅ、うぅ、うぅ、",
    "うぅ、うぅ、うぅ、うぅ、うぅ、うぅ、うぅ、",
    "うぅ、うぅ、うぅ、うぅ、うぅ、うぅ、うぅ、うぅ、",
    "うぅ、うぅ、うぅ、うぅ、うぅ、うぅ、うぅ、うぅ、うぅ、",
    "うぅーーー",
    "うぅぅ〜",
    "うぅぅ〜〜",
    "うぅぅ〜〜〜",
    "うぅぅ〜〜〜〜",
    "うぅぅ〜〜〜〜〜",
    "うぅぅ〜〜〜〜〜〜",
    "うぅぅ〜〜〜〜〜〜〜",
    "うぅぅ〜〜〜〜〜〜〜〜",
    "うぅぅ〜〜〜〜〜〜〜〜〜",
    "うぅぅう",
    "うぅぅう〜",
    "うぅぅう〜〜",
    "うぅぅう〜〜〜",
    "うぅぅう〜〜〜〜",
    "うぅぅう〜〜〜〜〜",
    "うぅぅう〜〜〜〜〜〜",
    "うぅぅう〜〜〜〜〜〜〜",
    "うぅぅう〜〜〜〜〜〜〜〜",
    "うぅぅう〜〜〜〜〜〜〜〜〜",
    "うぅぅうぅ",
    "うぅぅうぅ〜",
    "うぅぅうぅ〜〜",
    "うぅぅうぅ〜〜〜",
    "うぅぅうぅ〜〜〜〜",
    "うぅぅうぅ〜〜〜〜〜",
    "うぅぅうぅ〜〜〜〜〜〜",
    "うぅぅうぅ〜〜〜〜〜〜〜",
    "うぅぅうぅ〜〜〜〜〜〜〜〜",
    "うぅぅうぅ〜〜〜〜〜〜〜〜〜",
    "うぅぅうぅぅ〜",
    "うぅぅうぅぅ〜〜",
    "うぅぅうぅぅ〜〜〜",
    "うぅぅうぅぅ〜〜〜〜",
    "うぅぅうぅぅ〜〜〜〜〜",
    "うぅぅうぅぅ〜〜〜〜〜〜",
    "うぅぅうぅぅ〜〜〜〜〜〜〜",
    "うぅぅうぅぅ〜〜〜〜〜〜〜〜",
    "うぅぅうぅぅ〜〜〜〜〜〜〜〜〜",
    "うぅぅーーー",
    "うぅぅぅーーー",
    "うぅぅぅぅーーー",
    "うぅぅぅぅぅーーー",
    "うぅぅぅぅぅぅーーー",
    "うぅぅぅぅぅぅぅーーー",
    "うぅぅぅぅぅぅぅーー",
    "うぅぅぅぅぅぅぅー",
    "うぅぅぅぅぅぅぅ",
    "うぅぅぅぅぅ",
    "うぅぅぅぅぅぅ",
    "うぅぅぅぅぅぅぅ",
    "うぅぅぅぅぅぅぅぅ",
    "うぅぅぅぅぅっ",
    "うぅぅぅぅぅっっ",
    "うぅぅぅぅぅっっっ",
    "うぅぅぅぅぅっっっっ",
    "うぅぅぅぅぅっっっっっ",
    "うぅぅぅぅぅっっっっっっ",
    "うぅぅぅぅぅっっっっっっっ",
    "うぅぅぅぅぅっっっっっっっっ",
    "うぅぅぅぅぅっっっっっっっっっ",
    "うぃっ",
    "うぃっっ",
    "うぃっっっ",
    "うぃっっっっ",
    "うぃっっっっっ",
    "うぃっっっっっっ",
    "うぃっっっっっっっ",
    "うぃっっっっっっっっ",
    "うぃっっっっっっっっっ",
    "うぃっうぃっ",
    "うぃっうぃっうぃっ",
    "うぃっうぃっうぃっうぃっ",
    "うぃっうぃっうぃっうぃっうぃっ",
    "うぃっうぃっうぃっうぃっうぃっうぃっ",
    "うぃっうぃっうぃっうぃっうぃっうぃっうぃっ",
    "うぃっうぃっうぃっうぃっうぃっうぃっうぃっうぃっ",
    "あーーーーーーーーっっっっっっっ",
    "あーーーーーーーーっっっっっっ",
    "あーーーーーーーーっっっっっ",
    "あーーーーーーーーっっっっ",
    "あーーーーーーーーっっっ",
    "あーーーーーーーーっっ",
    "あーーーーーーーっっっっっ",
    "あーーーーーーーっっっっ",
    "あーーーーーーーっっっ",
    "あーーーーーーーっっ",
    "あーーーーーーーっ",
    "あーーーーーーーーーーーーー",
    "あーーーーーーーーーーーー",
    "あーーーーーーーーーーー",
    "あーーーーーーーーーー",
    "あーーーーーーーーー",
    "あーーーーーーーー",
    "あーーーーーーー",
    "あーーーーーー",
    "あーーーーー",
    "あーーーー",
    "あーーー",
    "あーー",
    "あー",
    "あー、",
    "あーん…",
    "あーんん…",
    "あーんんん…",
    "あーんんんん…",
    "あーんんんんん…",
    "あーんんんんんん…",
    "あーん",
    "あーんん",
    "あーんんん",
    "あーんんんん",
    "あーんんんんん",
    "あーんんんんんん",
    "あー、うー",
    "あー、うーー",
    "あー、うーーー",
    "あー、うーーーー",
    "あー、うーーーーー",
    "あー、うーーーーーー",
    "あー、うーーーーーーー",
    "あー、あー",
    "あー、あー、あー",
    "あー、あー、あー、あー",
    "あー、あー、あー、あー、あー",
    "あー、あー、あー、あー、あー、あー",
    "あー、あー、あー、あー、あー、あー、あー",
    "あー、あー、あー、あー、あー、あー、あー、あー",
    "あーあー",
    "あーあーあー",
    "あーあーあーあー",
    "あーあーあーあーあー",
    "あーあーあーあーあーあー",
    "あーあーあーあーあーあーあー",
    "あーあーあーあーあーあーあーあー",
    "あーあーあーあーあーあーあーあーあー",
    "あーあーあーあーあーあーあーあーあーあー",
    "あーあーあーあーあーあーあーあーあーあーあー",
    "あーあーあーあーあーあーあーあーあーあーあーあー",
    "あーー、うー",
    "あーーー、うー",
    "あーーーー、うー",
    "あーーーーー、うー",
    "あーー、うーー",
    "あーーー、うーーー",
    "あーーーー、うーーーー",
    "うーーん",
    "うーーん、",
    "うーそー",
    "うーそーー",
    "うーそーーー",
    "うーそーーーー",
    "うーそーーーーー",
    "うーそーーーーーー",
    "うーそーーーーーーー",
    "うーそーーーーーーーー",
    "うーそーーーーーーーーー",
    "あーーい",
    "あーーい、",
    "あーー、",
    "あ…",
    "あ…あ…",
    "あ…あ…あ…",
    "あ…あ…あ…あ…",
    "あ…あ…あ…あ…あ…",
    "あ…あ…あ…あ…あ…あ…",
    "あ…あ…あ…あ…あ…あ…あ…",
    "あ…うん…",
    "あうんあうん",
    "あうんあうんあうん",
    "あうんあうんあうんあうん",
    "あうんあうんあうんあうんあうん",
    "あうんあうんあうんあうんあうんあうん",
    "あうんあうんあうんあうんあうんあうんあうん",
    "あうんあうんあうんあうんあうんあうんあうんあうん",
    "あうんあうんあうんあうんあうんあうんあうんあうんあうん",
    "あうんあうんあうんあうんあうんあうんあうんあうんあうんあうん",
    "あうん",
    "あうんん",
    "あうんんん",
    "あうんんんん",
    "あうんんんんん",
    "あうんんんんんん",
    "あうんんんんんんん",
    "あうんんんんんんんん",
    "あうんんんんんんんんん",
    "あうんんんんんんんんんん",
    "くい",
    "くいくい",
    "くいくいくい",
    "くいくいくいくい",
    "くいくいくいくいくい",
    "くいくいくいくいくいくい",
    "くいくいくいくいくいくいくい",
    "くいくいくいくいくいくいくいくい",
    "くいくいくいくいくいくいくいくいくい",
    "くいくいくいくいくいくいくいくいくいくい",
    "あー、にもじーの",
    "あーにもじーの",
    "うふ",
    "うふふ",
    "うふふふ",
    "うふふふふ",
    "うふふふふふ",
    "うふふふふふふ",
    "うふふふふふふふ",
    "うふふふふふふふふ",
    "うふふうふふ",
    "うふふうふふうふふ",
    "うふふっ…",
    "うふふっ",
    "うふふっっ",
    "うふふっっっ",
    "うふふっっっっ",
    "うふふっっっっっ",
    "うふふっっっっっっ",
    "うふふっっっっっっっ",
    "うふふっっっっっっっっ",
    "うふふっっっっっっっっっ",
    "うぅぅ…",
    "うぅぅ",
    "うぅぅっ",
    "うぅぅっっ",
    "うぅぅっっっ",
    "うぅぅっっっっ",
    "うぅぅっっっっっ",
    "うぅぅっっっっっっ",
    "うぅぅっ…",
    "うぅぅっ…うぅぅっ",
    "うぅぅっ…うぅぅっ…",
    "うぅぅっ…うぅぅっ…うぅぅっ",
    "うぅぅっ…うぅぅっ…うぅぅっ…",
    "うぅっ",
    "うぅっっ",
    "うぅっっっ",
    "うぅっっっっ",
    "うぅっっっっっ",
    "うぅっっっっっっ",
    "うぅぅうぅぅ…",
    "うぅぅうぅぅ",
    "うぅぅぅ",
    "うぅぅぅぅ",
    "うぅうぅ",
    "うぅうぅうぅ",
    "うぅうぅうぅうぅ",
    "うぅうぅうぅうぅうぅ",
    "うぅうぅうぅうぅうぅうぅ",
    "うぅうぅうぅうぅうぅうぅうぅ",
    "うぅうぅうぅうぅうぅうぅうぅうぅ",
    "うぅうぅうぅうぅうぅうぅうぅうぅうぅ",
    "うぅうぅうぅうぅうぅうぅうぅうぅうぅうぅ",
    "うぅうぅぅ",
    "うぅう",
    "うぅうう",
    "うぅううう",
    "うぅうううう",
    "うぅううううう",
    "うぅうううううう",
    "うぅううううううう",
    "うぅうううううううう",
    "うぅううううううううう",
    "うぅうううううううううう",
    "うぅううううううううううう",
    "うぅうううううううううううあ",
    "うぅうううううううううううああ",
    "うぅうううううううううううあああ",
    "うぅ",
    "うぅ…うぅ…",
    "うぅ…うぅ…うぅ…",
    "うぅ…うぅ…うぅ…うぅ…",
    "うぅ…うぅ…うぅ…うぅ…うぅ…",
    "うぅ…うぅ…うぅ…うぅ…うぅ…うぅ…",
    "うぅ…うぅ…うぅ…うぅ…うぅ…うぅ…うぅ…",
    "うぅ…うぅ…うぅ…うぅ…うぅ…うぅ…うぅ…うぅ…",
    "うぅ…うぅ…うぅ…うぅ…うぅ…うぅ…うぅ…うぅ…うぅ…",
    "うぅ…うぅ…うぅ…うぅ…うぅ…うぅ…うぅ…うぅ…うぅ…うぅ…",
    "うぅ…",
    "うぅうぅ…",
    "うぅうぅうぅ…",
    "うぅうぅうぅうぅ…",
    "うぅうぅうぅうぅ",
    "うぅうぅうぅうぅうぅ",
    "うぅうぅうぅうぅうぅうぅ",
    "うぅうぅうぅうぅうぅうぅうぅ",
    "うぅうぅうぅうぅうぅうぅうぅうぅ",
    "うぅうぅうぅうぅうぅうぅうぅうぅうぅ",
    "うぅうぅうぅうぅうぅうぅうぅうぅうぅうぅ",
    "あっち",
    "うーん、はーーーーーーーーー",
    "うーん、はーーーーーーーー",
    "うーん、はーーーーーーー",
    "うーん、はーーーーーー",
    "うーん、はーーーーー",
    "うーん、はーーーー",
    "うーん、はーーー",
    "うーん、はーー",
    "うーん、はー",
    "うーん、はー、うーん、はー",
    "うーん、はー、うーん、はー、うーん、はー",
    "うーん、はー、うーん、はー、うーん、はー、うーん、はー",
    "うーん、はー、うーん、はー、うーん、はー、うーん、はー、うーん、はー",
    "うーん、はー、",
    "うーん、はー、うーん、はー、",
    "うーん、はー、うーん、はー、うーん、はー、",
    "うーん、はー、うーん、はー、うーん、はー、うーん、はー、",
    "うーん、はー、うーん、はー、うーん、はー、うーん、はー、うーん、はー、",
    "うーん",
    "うーん、",
    "うーん、うーん",
    "うーん、うーん、",
    "うーん、うーん、うーん",
    "うーん、うーん、うーん、",
    "うーん、うーん、うーん、うーん",
    "うーん、うーん、うーん、うーん、",
    "うーん、うーん、うーん、うーん、うーん",
    "うーん、うーん、うーん、うーん、うーん、",
    "うーん、うーん、うーん、うーん、うーん、うーん",
    "うーん、うーん、うーん、うーん、うーん、うーん、",
    "うーんうーん",
    "うーんうーんうーん",
    "うーんうーんうーんうーん",
    "うーんうーんうーんうーんうーん",
    "うーんうーんうーんうーんうーんうーん",
    "うーんうーんうーんうーんうーんうーんうーん",
    "うーんうーんうーんうーんうーんうーんうーんうーん",
    "うーんうーんうーんうーんうーんうーんうーんうーんうーん",
    "うーんうーんうーんうーんうーんうーんうーんうーんうーんうーん",
    "うーんうーんうーんうーんうーんうーんうーんうーんうーんうーんうーん",
    "うーん、あ、うーん、うーん、うーん、うーん",
    "うーん、あ、うーん、うーん、うーん、うーん、うーん",
    "うーん、あ、うーん、うーん、うーん、うーん、うーん、うーん",
    "うーん、あ、うーん、うーん、うーん、うーん、うーん、うーん、うーん",
    "うーん、あ、うーん、うーん、うーん、うーん、うーん、うーん、うーん、うーん",
    "うーんあうーん",
    "うーんあうーんうーん",
    "うーんあうーんうーんうーん",
    "うーんあうーんうーんうーんうーん",
    "うーんあうーんうーんうーんうーんうーん",
    "うーんあうーんうーんうーんうーんうーんうーん",
    "うーんあうーんうーんうーんうーんうーんうーんうーん",
    "うーんあうーんあうーん",
    "うーんあうーんあうーんうーん",
    "うーんあうーんあうーんうーんうーん",
    "うーんあうーんあうーんうーんうーんうーん",
    "うーんあうーんあうーんうーんうーんうーんうーん",
    "うーんあうーんあうーんうーんうーんうーんうーんうーん",
    "うーんあうーんあうーんうーんうーんうーんうーんうーんうーん",
    "あうーんうーん",
    "あうーんうーんうーん",
    "あうーんうーんうーんうーん",
    "あうーんうーんうーんうーんうーん",
    "あうーんうーんうーんうーんうーんうーん",
    "あうーんうーんうーんうーんうーんうーんうーん",
    "あうーんうーんうーんうーんうーんうーんうーんうーん",
    "あうーんうーんうーんうーんうーんうーんうーんうーんうーん",
    "あうーんうーんうーんうーんうーんうーんうーんうーんうーんうーん",
    "うー、あー",
    "うー、あー、あー",
    "うー、あー、あー、あー",
    "うー、あー、あー、あー、あー",
    "うー、あー、あー、あー、あー、あー",
    "うー、あー、あー、あー、あー、あー、あー",
    "うー、あー、あー、あー、あー、あー、あー、あー",
    "うー、あー、あー、あー、あー、あー、あー、あー、あー",
    "うーあー",
    "うーあーあー",
    "うーあーあーあー",
    "うーあーあーあーあー",
    "うーあーあーあーあーあー",
    "うーあーあーあーあーあーあー",
    "うーあーあーあーあーあーあーあー",
    "うーあーあーあーあーあーあーあーあー",
    "うーあーあーあーあーあーあーあーあーあー",
    "うーあーあーあーあーあーあーあーあーあーあー",
    "うーあーあーあーあーあーあーあーあーあーあーあー",
    "うーーあー",
    "うーーあーあー",
    "うーーあーあーあー",
    "うーーあーあーあーあー",
    "うーーあーあーあーあーあー",
    "うーーあーあーあーあーあーあー",
    "うーーあーあーあーあーあーあーあー",
    "うーーあーあーあーあーあーあーあーあー",
    "うーーあーあーあーあーあーあーあーあーあー",
    "うーーあーあーあーあーあーあーあーあーあーあー",
    "うーーあーあーあーあーあーあーあーあーあーあーあー",
    "うー",
    "うーうー",
    "うーうーうー",
    "うーうーうーうー",
    "うーうーうーうーうー",
    "うーうーうーうーうーうー",
    "うーうーうーうーうーうーうー",
    "うーうーうーうーうーうーうーうー",
    "うーうーうーうーうーうーうーうーうー",
    "うーうーうーうーうーうーうーうーうーうー",
    "うーうーうーうーうーうーうーうーうーうーうー",
    "わーーーーーーー",
    "わーーーーーー",
    "わーーーーー",
    "わーーーー",
    "わーーー",
    "わーー",
    "わー",
    "わぁー",
    "わぁーー",
    "わぁーーー",
    "わぁーーーー",
    "わぁーーーーー",
    "わぁーーーーーー",
    "わぁーーーーーーー",
    "はぁー",
    "はぁーー",
    "はぁーーー",
    "はぁーーーー",
    "はぁーーーーー",
    "はぁーーーーーー",
    "はぁー",
    "はぁーはぁー",
    "はぁーはぁーはぁー",
    "はぁーっ",
    "はぁーっっ",
    "はぁーっっっ",
    "はぁーっっっっ",
    "はぁーっっっっっ",
    "はぁーっっっっっっ",
    "はぁーっっっっっっっ",
    "はぁーっっっっっっっっ",
    "あ↑あ↑あ↓あ↓あ↑あ↓",
    "あはは",
    "あははは",
    "あはははは",
    "あははははは",
    "あはははははは",
    "あははははははは",
    "あはははははははは",
    "あははあはは",
    "あははあははあはは",
    "あははあははあははあはは",
    "あははあははあははあははあはは",
    "あははあははあははあははあははあはは",
    "あははあははあははあははあははあははあはは",
    "あははあははあははあははあははあははあははあはは",
    "あははあははあははあははあははあははあははあははあはは",
    "はあは",
    "はあはは",
    "はあははは",
    "はあはははは",
    "はあははははは",
    "はあはははははは",
    "はあははははははは",
    "はあはははははははは",
    "はあははははははははは",
    "はあはははははははははは",
    "はあはは",
    "はあはははあはは",
    "はあはははあはははあはは",
    "はあはははあはははあはははあはは",
    "はあはははあはははあはははあはははあはは",
    "はあはははあはははあはははあはははあはははあはは",
    "はあはははあはははあはははあはははあはははあはははあはは",
    "たべるし",
    "あははっ",
    "あははっっ",
    "あははっっっ",
    "あははっっっっ",
    "あははっっっっっ",
    "あははっっっっっっ",
    "あははっっっっっっっ",
    "あははっっっっっっっっ",
    "た、た、た、た、た",
    "た、た、た、た",
    "た、た、た",
    "た、た",
    "たたたたたたた",
    "たたたたたた",
    "たたたたた",
    "たたたた",
    "たたた",
    "たた",
    "っ",
    "っあ",
    "あはっ",
    "あはっっ",
    "あはっっっ",
    "あはっっっっ",
    "あはっっっっっ",
    "あはっっっっっっ",
    "あはっあはっ",
    "あはっあはっあはっ",
    "あはっあはっあはっあはっ",
    "あはっあはっあはっあはっあはっ",
    "あはっあはっあはっあはっあはっあはっ",
    "あはっあはっあはっあはっあはっあはっあはっ",
    "あはっあはっあはっあはっあはっあはっあはっあはっ",
    "あはっあはっあはっあはっあはっあはっあはっあはっあはっ",
    "あはっあはっあはっあはっあはっあはっあはっあはっあはっあはっ",
    "あ゛ぁあ゛あ゛あ゛あ゛あ゛",
    "あぁあ",
    "あぁああ",
    "あぁあああ",
    "あぁああああ",
    "あぁあああああ",
    "あぁああああああ",
    "あぁあああああああ",
    "あぁああああああああ",
    "あぁあああああああああ",
    "あ゛ぁぁぁぁぁぁぁぁぁぁぁ",
    "あ゛ぁぁぁぁぁぁぁぁぁぁ",
    "あ゛ぁぁぁぁぁぁぁぁぁ",
    "あ゛ぁぁぁぁぁぁぁぁ",
    "あ゛ぁぁぁぁぁぁぁ",
    "あ゛ぁぁぁぁぁぁ",
    "あ゛ぁぁぁぁぁ",
    "あ゛ぁぁぁぁ",
    "あ゛ぁぁぁ",
    "あ゛ぁぁ",
    "あ゛ぁ",
    "あ゛ぁぁぁぁぁぁぁぁ!!",
    "あ゛ぁぁぁぁぁぁぁ!!",
    "あ゛ぁぁぁぁぁぁ!!",
    "あ゛ぁぁぁぁぁ!!",
    "あ゛ぁぁぁぁ!!",
    "あ゛ぁぁぁ!!",
    "あ゛ぁぁ!!",
    "あ゛ぁ!!",
    "あ゛ぁぁぁぁぁぁぁぁ!",
    "あ゛ぁぁぁぁぁぁぁ!",
    "あ゛ぁぁぁぁぁぁ!",
    "あ゛ぁぁぁぁぁ!",
    "あ゛ぁぁぁぁ!",
    "あ゛ぁぁぁ!",
    "あ゛ぁぁ!",
    "あ゛ぁ!",
    "あ゛あ゛あ゛あ゛あ゛あ゛",
    "あ゛あ゛あ゛あ゛あ゛",
    "あ゛あ゛あ゛あ゛",
    "あ゛あ゛あ゛あ゛…",
    "あ゛あ゛あ゛",
    "あ゛あ゛",
    "あ゛",
    "わぁあ あ",
    "わぁあ ああ",
    "わぁあ あああ",
    "わぁあ ああああ",
    "わぁあ あああああ",
    "わぁあ ああああああ",
    "わぁあ あああああああ",
    "わぁあ ああああああああ",
    "わぁあ あああああああああ",
    "わぁあ ああああああああああ"
    "あ゛あ゛~~~~~~~~",
    "あ゛あ゛~~~~~~~",
    "あ゛あ゛~~~~~~",
    "あ゛あ゛~~~~~",
    "あ゛あ゛~~~~",
    "あ゛あ゛~~~",
    "あ゛あ゛~~",
    "あ゛あ゛~",
    "あ゛あ~~~~~~~~",
    "あ゛あ~~~~~~~",
    "あ゛あ~~~~~~",
    "あ゛あ~~~~~",
    "あ゛あ~~~~",
    "あ゛あ~~~",
    "あ゛あ~~",
    "あ゛あ~",
    "あ゛~",
    "あ゛~~",
    "あ゛~~~",
    "あ゛~~~~",
    "あ゛~~~~~",
    "あ゛~~~~~~",
    "あ゛~~~~~~~",
    "あ゛~~~~~~~~",
    "あ゛~~~~~~~~~",
    "あ゛ぁ゛あ゛あ゛あ゛あ゛あ",
    "あ゛ぁ゛あ゛あ",
    "あ゛ぁ",
    "あ゛",
    "あ゛ぁ゛",
    "あ゛ぁ゛あ゛",
    "あ゛ぁ゛あ゛あ゛",
    "あ゛ぁ゛あ゛あ゛あ゛",
    "あ゛ぁ゛あ゛あ゛あ゛あ゛",
    "あ゛ぁ゛あ゛あ゛あ゛あ゛あ゛",
    "あ゛ぁ゛あ゛あ゛あ゛あ゛あ゛あ゛",
    "あ゛あ゛あ゛あ゛あ゛あ゛あ゛!",
    "あ゛あ゛あ゛あ゛あ゛あ゛あ゛あ",
    "あ゛あ゛あ゛あ゛あ゛あ゛あ゛あ゛",
    "あ゛あ゛あ゛あ゛あ゛あ゛あ゛",
    "あ゛ぁ゛ぁ゛あ゛ぁ゛ぁあ゛ぁ゛ぁ",
    "あ゛ぁ゛ぁ゛あ゛ぁ゛ぁ゛あ゛ぁ゛ぁ",
    "あ゛ぁ゛ぁ゛あ゛",
    "あ゛ぁ゛ぁ゛あ゛ぁ゛",
    "あ゛ぁ゛ぁ゛あ゛ぁ゛ぁ",
    "あ゛ぁ゛ぁ゛ぁ゛ぁ゛ぁ",
    "あ゛ぁ゛ぁ゛ぁ゛ぁ",
    "あ゛ぁ゛ぁ゛ぁ",
    "あ゛ぁ゛ぁ゛",
    "あ゛ぁ゛ぁ",
    "あ゛ぁ゛",
    "あ゛ぁ…",
    "あ゛ぁ…うぅ",
    "あ゛ぁうぅ",
    "あ゛ぁうぅ…",
    "あ゛ぁ…うぅ…",
    "あ゛ぁ…うぅ…あ゛ぁ…うぅ…",
    "あ゛ぁ…うぅ…あ゛ぁ…うぅ…あ゛ぁ…うぅ…",
    "あ゛ぁあ゛ぁ",
    "あ゛ぁあ゛ぁ…",
    "あ゛ぁぁ",
    "あ゛ぁぁ…",
    "うぅうぅう",
    "うーっ",
    "ああっ!",
    "ああっ",
    "ああっっ",
    "ああっっっ",
    "ああっっっっ",
    "ああっっっっっ",
    "ああっっっっっっ",
    "ああっっっっっっっ",
    "ああっっっっっっっっ",
    "ああっっっっっっっっっ",
    "ああー",
    "あーっ",
    "あーっっ",
    "あーっっっ",
    "あーっっっっ",
    "あーっっっっっ",
    "あーっっっっっっ",
    "あーっっっっっっっ",
    "あーっっっっっっっっ",
    "あーっっっっっっっっっ",
    "あーっっっっっっっっっっ",
    "あーっっっっっっっっっっっ",
    "あーっあーっ",
    "あーっあーっあーっ",
    "あーっあーっあーっあーっ",
    "あーっあーっあーっあーっあーっ",
    "あーっあーっあーっあーっあーっあーっ",
    "あーっあーっあーっあーっあーっあーっあーっ",
    "あーっあーっあーっあーっあーっあーっあーっあーっ",
    "あーっあーっあーっあーっあーっあーっあーっあーっあーっ",
    "あーっあーっあーっあーっあーっあーっあーっあーっあーっあーっ",
    "あーっあーっあーっあーっあーっあーっあーっあーっあーっあーっあーっ",
    "あ゛っ",
    "うぅん",
    "うぅんん",
    "うぅんんん",
    "うぅんんんん",
    "うぅんんんんん",
    "うぅんんんんんん",
    "うぅんんんんんんん",
    "うぅんんんんんんんん",
    "うぅんんんんんんんんん",
    "うぅんんんんんんんんんん",
    "うぅん、",
    "うぅん、うぅん",
    "うぅん、うぅん、",
    "うぅん、うぅん、うぅん",
    "うぅん、うぅん、うぅん、",
    "うぅん、うぅん、うぅん、うぅん",
    "うぅん、うぅん、うぅん、うぅん、",
    "うぅん、うぅん、うぅん、うぅん、うぅん",
    "うぅん、うぅん、うぅん、うぅん、うぅん、",
    "うぅん、うぅん、うぅん、うぅん、うぅん、うぅん",
    "うぅん、うぅん、うぅん、うぅん、うぅん、うぅん",
    "うぅん、うぅん、うぅん、うぅん、うぅん、うぅん",
    "あはー",
    "あはーー",
    "あはーーー",
    "あはーーーー",
    "あはーーーーー",
    "あはーーーーーー",
    "あはーーーーーーー",
    "あはーーーーーーーー",
    "あはーーーーーーーーー",
    "あはーあはー",
    "あはーあはーあはー",
    "あはーあはーあはーあはー",
    "あはーあはーあはーあはーあはー",
    "あはーあはーあはーあはーあはーあはー",
    "あはーあはーあはーあはーあはーあはーあはー",
    "あはーあはーあはーあはーあはーあはーあはーあはー",
    "あはーあはーあはーあはーあはーあはーあはーあはーあはー",
    "あはーあはーあはーあはーあはーあはーあはーあはーあはーあはー",
    "あはーあはーあはーあはーあはーあはーあはーあはーあはーあはーあはー",
    "あへー",
    "あへーー",
    "あへーーー",
    "あへーーーー",
    "あへーーーーー",
    "あへーーーーーー",
    "あへーーーーーーー",
    "あへーーーーーーーー",
    "あへーーーーーーーーー",
    "3,2,1",
    "ペコペコ",
    "へー",
    "へーー",
    "へーーー",
    "へーーーー",
    "へーーーーー",
    "へーーーーーー",
    "へーーーーーーー",
    "へーーーーーーーー",
    "へーーーーーーーーー",
    "へーーーーーーーーーー",
    "へーーーーーーーーーーー",
    "あへー",
    "あへーあへー",
    "あへーあへーあへー",
    "あへーあへーあへーあへー",
    "あへーあへーあへーあへーあへー",
    "あへーあへーあへーあへーあへーあへー",
    "あへーあへーあへーあへーあへーあへーあへー",
    "あへーあへーあへーあへーあへーあへーあへーあへー",
    "あへーあへーあへーあへーあへーあへーあへーあへーあへー",
    "うぅんっ",
    "うぅんっっ",
    "うぅんっっっ",
    "うぅんっっっっ",
    "うぅんっっっっっ",
    "うぅんっっっっっっ",
    "うぅんっっっっっっっ",
    "うぅんっっっっっっっっ",
    "うぅんっっっっっっっっっ",
    "うぅんっっっっっっっっっっ",
    "あっ、",
    "あっ、あっ",
    "あっ、あっ、あっ",
    "あっ、あっ、あっ、あっ",
    "あっ、あっ、あっ、あっ、あっ",
    "あっ、あっ、あっ、あっ、あっ、あっ",
    "あっ、あっ、あっ、あっ、あっ、あっ、あっ",
    "あっ、あっ、あっ、あっ、あっ、あっ、あっ、あっ",
    "あっ",
    "あっあっ",
    "あっあっあっ",
    "あっあっあっあっ",
    "あっあっあっあっあっ",
    "あっあっあっあっあっあっ",
    "あっあっあっあっあっあっあっ",
    "あっあっあっあっあっあっあっあっ",
    "あいっ、あいっ、あいっ",
    "あいっ",
    "あいっあいっ",
    "あいっあいっあいっ",
    "あいっあいっあいっあいっ",
    "あいっあいっあいっあいっあいっ",
    "あいっあいっあいっあいっあいっあいっ",
    "あいっあいっあいっあいっあいっあいっあいっ",
    "あいっ!",
    "あいっあいっ!",
    "あいっあいっあいっ!",
    "あいっあいっあいっあいっ!",
    "あいっあいっあいっあいっあいっ!",
    "あいっあいっあいっあいっあいっあいっ!",
    "あいっあいっあいっあいっあいっあいっあいっ!",
    "あいっ! んっ!",
    "あいっ! んっんっ!",
    "あいっ! んっんっんっ!",
    "あいっ! んっんっんっんっ!",
    "あいっ! んっんっんっんっんっ!",
    "あい",
    "あん",
    "あんん",
    "あんんん",
    "あんんんん",
    "あんんんんん",
    "あんんんんんん",
    "あんんんんんんん",
    "あんんんんんんんん",
    "あんんんんんんんんん",
    "はぁ…はぁ…はぁ…はぁ…はぁ…",
    "はぁ…はぁ…はぁ…はぁ…",
    "はぁ…はぁ…はぁ…",
    "はぁ…はぁ",
    "はぁ…はぁ…",
    "は~~~~~~~~",
    "は~~~~~~~",
    "は~~~~~~",
    "は~~~~~",
    "は~~~~",
    "は~~~",
    "は~~",
    "は~",
    "は",
    "はは",
    "ははは",
    "はははは",
    "ははははは",
    "はははははは",
    "ははははははは",
    "ー",
    "ーー",
    "ーーー",
    "ーーーー",
    "ーーーーー",
    "ーーーーーー",
    "ーーーーーーー",
    "ーーーーーーーー",
    "ーーーーーーーーー",
    "ーーーーーーーーーー",
    "ーーーーーーーーーーー",
    "ーーーーーーーーーーーー",
    "ーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー",
    "ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー",
    "あ゛ー!",
    "あ゛ー",
    "おばあちゃん",
    "※この動画の字幕は視聴者によって作成されました。",
    "※この動画の字幕は視聴者によって作成されました",
    "この動画の字幕は視聴者によって作成されました。",
    "この動画の字幕は視聴者によって作成されました",
    "a",
    "aa",
    "ah",
    "ahh",
    "ha",
    "haa",
    "hah",
    "haha",
    "hahaha",
    "mmm",
    "mm",
    "m",
    "h",
    "o",
    "mh",
    "mmh",
    "hm",
    "hmm",
    "huh",
    "oh",
]

need_context_lines = [
    "feelsgod",
    "godbye",
    "godnight",
    "thankyou",
]



# Define a generator function to generate all audio files with no srt in a directory
def audio_files_generator(directory_path):
    audio_files = []
    for file_name in os.listdir(directory_path):
        if file_name.endswith((".mp3", ".wav", ".aac", ".ogg", ".flac")):
            # Check if corresponding .srt file exists
            base_name = os.path.splitext(file_name)[0]  # Get the base name of the file (without extension)
            srt_file = base_name + ".srt"  # Corresponding .srt file name
            if not os.path.exists(os.path.join(directory_path, srt_file)):  # If .srt file does not exist
                audio_files.append(os.path.join(directory_path, file_name))
    # Sort the files in alphabetical order
    audio_files.sort()
    for file_name in audio_files:
        yield file_name



# Usage:
# clean_srt_file('path_to_your_srt_file', 'path_to_target_directory', ['list_of_hallucination_words_1', 'list_of_hallucination_words_2', 'list_of_hallucination_words_3'])
def clean_srt_file(source_file, target_dir, hallucination_lists):
    # Load the SRT file
    subs = pysrt.open(source_file, encoding='utf-8')

    # Use hallucination words from provided lists
    hallucination_words = [word for sublist in hallucination_lists for word in sublist]

    i = 0
    while i < len(subs):
        # Merge consecutive identical lines
        if i < len(subs) - 1 and subs[i].text == subs[i+1].text and (subs[i+1].start - subs[i].end).seconds < 0.4:
            subs[i].end = subs[i+1].end
            del subs[i+1]
            continue


        # This pattern will match any phrase (sequence of words between commas) that is repeated at least 2 times
        text= subs[i].text
        pattern = r"((?:[^,]*,)\s*)(\1{1,})"
        while re.search(pattern, text):
            text = re.sub(pattern, r"\1", text)
            subs[i].text = text

        # Remove repeated words in a line
        words = subs[i].text.split()
        j = 0
        while j < len(words) - 2:
            if words[j] == words[j+1] == words[j+2]:
                del words[j]
            else:
                j += 1
        subs[i].text = ' '.join(words)

        # Remove hallucination words or sentences
        for word in hallucination_words:
            word_regex = r'(?<=\s)' + re.escape(word) + r'(?=\s)'
            subs[i].text = re.sub(word_regex, '', ' ' + subs[i].text + ' ').strip()

        # Adjust duration of long subtitle lines
        if (subs[i].end - subs[i].start).seconds > 6:
            num_chars = len(subs[i].text)
            new_duration = num_chars / 15.0  # assuming 15 characters per second

            start_time = datetime.datetime.combine(datetime.date.today(), subs[i].start.to_time())
            new_end_time = start_time + datetime.timedelta(seconds=new_duration)

            subs[i].end.hours = new_end_time.hour
            subs[i].end.minutes = new_end_time.minute
            subs[i].end.seconds = new_end_time.second
            subs[i].end.milliseconds = new_end_time.microsecond // 1000

        i += 1

    # Remove empty subtitle lines and renumber the SRT file
    subs_cleaned = [s for s in subs if s.text.strip() != '']
    for i, sub in enumerate(subs_cleaned):
        sub.index = i + 1

    # Check if target directory exists, if not create it
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    # Save cleaned SRT file to target directory
    base_name = os.path.basename(source_file)
    new_name = os.path.splitext(base_name)[0] + '_cleaned.srt'
    target_file = os.path.join(target_dir, new_name)

    with open(target_file, 'w', encoding='utf-8') as f:
        for sub in subs_cleaned:
            f.write(str(sub))
            f.write('\n\n')

# Usage:
# clean_srt_file('path_to_your_srt_file', 'path_to_target_directory', ['list_of_hallucination_words_1', 'list_of_hallucination_words_2', 'list_of_hallucination_words_3'])
def clean_srt_file_japanese(source_file, target_dir):
    # Load the SRT file
    subs = list(srt.parse(open(source_file, 'r', encoding='utf-8').read()))

    i = 0
    while i < len(subs):
        # Merge consecutive identical lines
        if i < len(subs) - 1 and subs[i].content == subs[i+1].content and (subs[i+1].start - subs[i].end).seconds < 0.4:
            subs[i].end = subs[i+1].end
            del subs[i+1]
            continue

        text = subs[i].content

        pattern = r"([^\s]+)(\s*\1){3,}"
        while re.search(pattern, text):
          text = re.sub(pattern, r"\1", text)


        if text != subs[i].content:
            # Calculate new start time if line was changed
            duration = len(text) * 0.040 * 1000 #miliseconds

            start_time = subs[i].start
            end_time = start_time + timedelta(milliseconds=duration)

            new_sub = srt.Subtitle(i+1, start_time, end_time, text)

            subs[i] = new_sub


       # Adjust duration of long subtitle lines
        if (subs[i].end - subs[i].start).seconds > 6:
            num_chars = len(subs[i].content)
            new_duration = num_chars / 15.0  # assuming 15 characters per second

            #start_time = datetime.datetime.combine(datetime.date.today(), subs[i].start.to_time())
            #new_end_time = start_time + datetime.timedelta(seconds=new_duration)

            start_time = subs[i].start
            new_end_time = start_time + timedelta(seconds=new_duration)
            subs[i].end = new_end_time

            #subs[i].end.hours = new_end_time.hour
            #subs[i].end.minutes = new_end_time.minute
            #subs[i].end.seconds = new_end_time.second
            #subs[i].end.milliseconds = new_end_time.microsecond // 1000




        i += 1

    # Remove empty subtitle lines and renumber the SRT file
    subs_cleaned = [s for s in subs if s.content.strip() != '']
    for i, sub in enumerate(subs_cleaned):
        sub.index = i + 1

    # Check if target directory exists, if not create it
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    # Save cleaned SRT file to target directory
    base_name = os.path.basename(source_file)
    new_name = os.path.splitext(base_name)[0] + '_cleaned.srt'
    target_file = os.path.join(target_dir, new_name)

    with open(target_file, 'w', encoding='utf-8') as f:
        f.write(srt.compose(subs_cleaned))

# Usage:
# clean_srt_file_japanese('path_to_your_srt_file', 'path_to_target_directory')



def punctuateText(inputtext, modelPath):
    if not isinstance(inputtext, str):
        print("Error: Input text is not a valid string.")
        return inputtext  # Return the input text unchanged or handle the error appropriately
    print("\n === inputtext is: ", inputtext)


    # Rest of your code

    base_name = "model_to_add_punctuation"

    tagger = nagisa.Tagger(
        vocabs=f"{base_name}.vocabs",
        params=f"{base_name}.params",
        hp=f"{base_name}.hp",
    )

    tokens = tagger.tagging(inputtext)

    words = tokens.words
    tags = tokens.postags

    outputs = []
    for word, tag in zip(words, tags):
        outputs.append(word)
        if tag == "1":
            outputs.append("、")
        elif tag == "2":
            outputs.append("。")

    # 突然ですがこれは日本語句読点付与プログラムです試してみてください
    # print(inputtext)

    # 突然ですが、これは日本語句読点付与プログラムです。試してみてください。
    print("".join(outputs))
    return "".join(outputs)



def sanitiseHallucinationEN(segs, modelPath):
  return segs





def sanitiseHallucinationJA(segs, modelPath):


    tseg = copy.deepcopy(segs)
    print("\n === tseg text is: ", tseg.text)
    tseg.text = tseg.text + "test"


    # Remove repetition
    newtext = tseg.text
    pattern = r"([^\s]+)(\s*\1){3,}"
    while re.search(pattern, newtext):
        newtext = re.sub(pattern, r"\1", newtext)

    if segs.text != newtext:
        # The text was changed
        duration = len(tseg.text) * 0.040
        segs.end = tseg.start + duration
        tseg.text = newtext
        tseg.text += ' 。 (cut)。'

    # punctuate the text
    tseg.text = punctuateText(tseg.text, modelPath)



    # if no_speech_prob then append possible Hallu
    if segs.no_speech_prob > 0.8:
        tseg.text += ' 。(hallu)。'

    return tseg


def testsanitiseHallucinationJA(segs, modelPath):

    print("sanitiseHallucinationJA function has been called")  # New print statement
    print(f"segs: {segs}")  # This will print the value of 'segs'

    print(type(segs))  # This will print the type of 'segs'
    if isinstance(segs, dict):  # This checks if 'segs' is a dictionary
        print(segs.keys())  # If 'segs' is a dictionary, this will print its keys

    if isinstance(segs, dict) and 'segments' in segs:
        for i, segment in enumerate(segs['segments']):
            print(f"Segment {i}: {type(segment)}")
            if isinstance(segment, dict):
                print(f"Keys in segment {i}: {segment.keys()}")
    return segs




# Main code where the magic happens
def run_whisper_wrapper(
    model_size = "large",
    audio_path = "/content/drive/MyDrive/WhisperJAV",
    language = "japanese",
    task = "translate",
    vad_threshold = 0.4,
    chunk_duration = 15.0,
    new_vocabulary = "",
    deepl_flag = False,
    deepl_target_lang = "EN-US",
    deepl_authkey = "",
    condition_on_previous_text = False
    ):

    # some sanity checks
    assert vad_threshold >= 0.01
    assert chunk_duration >= 0.1
    assert audio_path != ""
    assert language != ""
    language = language.lower()
    assert language in TO_LANGUAGE_CODE, "invalid language"


    if new_vocabulary.strip() == "":
      new_vocabulary = None

    print(str(deepl_flag))



    # Get the base file name without extension
    audiofilebasename = os.path.basename(os.path.splitext(audio_path)[0])
    subfolder = ".subs.original"

    # Get the directory of the audio file
    audiofile_dir = os.path.dirname(audio_path)

    # Construct the subfolder path
    subfolder_path = os.path.join(audiofile_dir, subfolder)

    # Create the subfolder if it doesn't exist
    os.makedirs(subfolder_path, exist_ok=True)

    if (task=="translate"):
      out_path = os.path.join(subfolder_path, audiofilebasename + "-en" + ".srt")
      out_path_pre = os.path.join(subfolder_path, audiofilebasename + "-en" + "_original.srt")
    else:
      out_path = os.path.join(subfolder_path, audiofilebasename + "-ja" + ".srt")
      out_path_pre = os.path.join(subfolder_path, audiofilebasename + "-ja" + "_original.srt")


    print("Running Whisper … PLEASE WAIT")
    segments, info = WHISPER_MODEL.transcribe(
      audio_path,
      task=task,
      language=TO_LANGUAGE_CODE[language],
      condition_on_previous_text=condition_on_previous_text,
      initial_prompt=new_vocabulary,
      word_timestamps=True,
      temperature=0,
      beam_size=2,
      best_of=2,
      patience=2,
      vad_filter=True,
      vad_parameters=dict(threshold=vad_threshold, max_speech_duration_s=chunk_duration),
    )


    PROTOFLAG = False
    absfolderpath = os.path.dirname(os.path.abspath(audio_path))

    subs = []
    segment_info = []
    timestamps = 0.0  # for progress bar

    #with tqdm(total=info.duration, bar_format=TQDM_FORMAT) as pbar:
    with tqdm(total=info.duration) as pbar:
      for i, seg in enumerate(segments, start=1):
        # Keep segment info for debugging
        segment_info.append(seg)

        if PROTOFLAG == True :

            # make a deep copy of each entry
            temp_seg = copy.deepcopy(seg)

            # Clean up possible hallucination
            if (task == "transcribe"):
                temp_seg = sanitiseHallucinationJA(temp_seg, absfolderpath)
            elif (task == "translate"):
                temp_seg = sanitiseHallucinationEN(temp_seg, absfolderpath)
            else:
                print("\n ==== sanitisation funtion NOT performed! ==== \n")


        # Add to SRT list
        subs.append(srt.Subtitle(
          index=i,
          start=datetime.timedelta(seconds=seg.start),
          end=datetime.timedelta(seconds=seg.end),
          content=seg.text.strip(),
        ))
        pbar.update(seg.end - timestamps)
        timestamps = seg.end
      if timestamps < info.duration:
        pbar.update(info.duration - timestamps)

    #for debugging only
    #with open("segment_info.debug.json", mode="w", encoding="utf8") as f:
    #  json.dump(segment_info, f, indent=4)

    # DeepL translation
    translate_error = False
    if deepl_flag:
      print("Translating with DeepL …")
      with open(out_path_pre, "w", encoding="utf8") as f:
        f.write(srt.compose(subs))
      print("(Japanese original subs saved to", out_path_pre, ")")

      lines = []
      for i in range(len(subs)):
        if language == "japanese":
          if subs[i].content[-1] not in PUNCT_MATCH:
            subs[i].content += "。"
          subs[i].content = "「" + subs[i].content + "」"
        else:
          if subs[i].content[-1] not in PUNCT_MATCH:
            subs[i].content += "."
          subs[i].content = '"' + subs[i].content + '"'
      for i in range(len(subs)):
        lines.append(subs[i].content)

      grouped_lines = []
      english_lines = []
      for i, l in enumerate(lines):
        if i % 30 == 0:
          # Split lines into smaller groups, to prevent error 413
          grouped_lines.append([])
          if i != 0:
            # Include previous 3 lines, to preserve context between splits
            grouped_lines[-1].extend(grouped_lines[-2][-3:])
        grouped_lines[-1].append(l.strip())

      try:
        translator = deepl.Translator(deepl_authkey)
        for i, n in enumerate(tqdm(grouped_lines)):
          x = ["\n".join(n).strip()]
          if language == "japanese":
            result = translator.translate_text(x, source_lang="JA", target_lang=deepl_target_lang)
          else:
            result = translator.translate_text(x, target_lang=deepl_target_lang)
          english_tl = result[0].text.strip().splitlines()
          assert len(english_tl) == len(n), f"Invalid translation line count ({len(english_tl)} vs {len(n)})"
          if i != 0:
            english_tl = english_tl[3:]
          for e in english_tl:
            english_lines.append(
              e.strip().translate(REMOVE_QUOTES).replace("’", "'")
            )
        for i, e in enumerate(english_lines):
          subs[i].content = e
      except Exception as e:
        print("DeepL translation error:", e)
        print("(downloading untranslated version instead)")
        translate_error = True

    # Write SRT file
    if translate_error:
      g_files.download(out_path_pre)
    else:
      # Removal of garbage lines
      clean_subs = []
      last_line_garbage = False
      for i in range(len(subs)):
        c = clean_text(subs[i].content)
        is_garbage = True
        for w in c.split(" "):
          w_tmp = w.strip()
          if w_tmp == "":
            continue
          if w_tmp in GARBAGE_LIST2:
            continue
          elif w_tmp in NEED_CONTEXT_LINES and last_line_garbage:
            continue
          else:
            is_garbage = False
            break
        if not is_garbage:
          clean_subs.append(subs[i])
        last_line_garbage = is_garbage
      with open(out_path, mode="w", encoding="utf8") as f:
        f.write(srt.compose(clean_subs))
      print("\nDone! Subs written to", out_path)


      if (task == "translate") :
        clean_srt_file(out_path, ".", [garbage_list, suppress_low, suppress_high])

      elif (task == "transcribe") and (deepl_flag):
        clean_srt_file(out_path_pre, ".", [garbage_list, suppress_low, suppress_high])
        clean_srt_file_japanese(out_path, ".")

      elif (task == "transcribe") and (not deepl_flag):
        clean_srt_file_japanese(out_path, ".")

      else:
	      raise ValueError("Oops Something has gone wrong. Missing modes")


      print("Downloading SRT file:")
      g_files.download(out_path)


In the next step you will be asked to connect your Google Drive.

After connecting your Drive you need to upload your audio files to your drive and to enter the path to that folder into the form below: "audio_folder".

Note: the default folder where the script looks for audio files is WhisperJAV folder in your root Drive directory.

Note: To get the path to your folder, click on folder icon on left side-bar of the Colab. Then click on "drive | MyDrive" and proceed deeper into your target folder. You can copy the path to your target folder by right-click on the folder, select path. Paste that path in the form below: audio_folder.

Note: all audio files in that folder will be subbed.

The SRT results of each file are saved in the same folder where the audio files are. When the script finishes subbing all audio files, it zipps all SRTs in one zip file and downloads it to your computer.



In [ ]:
#@markdown **Mount Google Drive**
from google.colab import drive
drive.mount('drive')

In [ ]:
#@markdown **Enter path to the folder where your audio files are in Google drive.**

#@markdown **Default location for audio files is WhisperJAV.



audio_folder = "/content/drive/MyDrive/WhisperJAV"  # @param {type:"string"}

if audio_folder == "" :
  raise ValueError("Folder path cannot be empty. Please input: /content/drive/MyDrive/WhisperJAV" )


In [ ]:
#@markdown **Runs Whisper on all audio files in the given folder**

#@markdown You can select among options from pulldown menus below:

#@markdown ========================================



import torch, torchaudio, os, srt, datetime, json, faster_whisper
from tqdm import tqdm
import pysrt
import re
from datetime import timedelta
from google.colab import files as g_files, drive as g_drive
import nagisa
import copy
import deepl





select_model = "large-v2"  # @param ["tiny","base","small","medium", "large-v1", "large-v2"]
translation_mode = "Japanese + English with DeepL"  # @param ["Japanese transcription", "English translation", "Japanese + English with DeepL"]
vad_threshold = 0.3  # @param {type:"number"}
chunk_duration = 5.0  # @param {type:"number"}
#@markdown If you've selected DeepL then give your auth key:
deepl_authkey = "031b44f4-7bdf-408b-0e9c-274466243b7d:fx"  # @param {type:"string"}

#@markdown ========================================


# Inputs sanity checks
assert vad_threshold >= 0.01
assert chunk_duration >= 0.1


#DEFAULTS
audio_language = "japanese"
task = "transcribe"
initial_prompt = ""
previous_text = False
run_deepl = False
deepl_target_lang = "EN-US"


audio_files_list = list(audio_files_generator(audio_folder))
WHISPER_MODEL = faster_whisper.WhisperModel(select_model, device="cuda")


if translation_mode == "English translation":
    task = "translate"
    run_deepl = False
elif translation_mode == "Japanese + English with DeepL":
    task = "transcribe"
    run_deepl = True
    if deepl_authkey == "" :
        raise ValueError("You have selected DeepL but DeepL Auth Key is missing. Please input DeepL Auth Key and Run all again")
elif translation_mode == "Japanese transcription":
  	task = "transcribe"
  	run_deepl = False
else:
	raise ValueError("Invalid translation mode")




# use the %cd magic command to change the directory to audio_dir
os.chdir(audio_folder)
%cd $audio_folder
!pwd
#!ls -la

# Call the process_audio_file function for each audio file in the list
for audio_file in audio_files_list:
    print("\nProcessing file: ")
    print(audio_file)
    run_whisper_wrapper(
        select_model,
        audio_file,
        audio_language,
        task,
        vad_threshold,
        chunk_duration,
        initial_prompt,
        run_deepl,
        deepl_target_lang,
        deepl_authkey,
        previous_text
    )


print("all done!")




In [ ]:
#@markdown **Download all subs in the folder as a zip file**
!zip -r subs.zip . -i *.srt
from google.colab import files
files.download('subs.zip')
!pwd
!ls -la

===========================================================

# ***  Credits, instructions, and coming features ***


============================================================

Credits: @Anon_entity, @phineas-pta, JAV communities Scanlover and Akiba

Upcoming features: Making cleaned SRTs default output + Adding DeepL.

For any comments, feature requests reach me at meizhong.986@gmail.com or in the scanlover forum

**Version History:**

0.3: 1st public release

0.4: Changed UI and default folder

0.5: Added SRT post processing. Cleaned SRTs are in .cleaned.subs subfolder
